#Setting Up Spark

#Books:

Learn PySpark: Build Python-based Machine Learning and Deep Learning Models by Promod Singh/ Apress

In [1]:
!git clone https://github.com/Apress/learn-pyspark.git
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://mirrors.estointernet.in/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf '/content/spark-3.1.2-bin-hadoop3.2.tgz'
!pip install -q findspark

Cloning into 'learn-pyspark'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 44 (delta 11), reused 44 (delta 11), pack-reused 0
Unpacking objects: 100% (44/44), done.


In [2]:
#Spark env seyup



import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"


import findspark
findspark.init()
print('spark location -->', findspark.find())

spark location --> /content/spark-3.1.2-bin-hadoop3.2


In [82]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import pandas_udf,PandasUDFType
spark = SparkSession.builder.master("local[*]").appName('atom').getOrCreate()


In [11]:
#Creating Dataframe
schema = StructType().add("user_id","string").add("country","string").add("browser","string").add("os","string").add("age","integer")

In [12]:
df = spark.createDataFrame([('A201','India','Chrome','WIN',33),('A202','UK','Safari','MAC',27),
                            ('A203','US','Firefox','UBUNTU',2)],schema=schema)

In [ ]:
df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- country: string (nullable = true)
 |-- browser: string (nullable = true)
 |-- os: string (nullable = true)
 |-- age: integer (nullable = true)



In [ ]:
df.show()

+-------+-------+-------+------+---+
|user_id|country|browser|    os|age|
+-------+-------+-------+------+---+
|   A201|  India| Chrome|   WIN| 33|
|   A202|     UK| Safari|   MAC| 27|
|   A203|     US|Firefox|UBUNTU|  2|
+-------+-------+-------+------+---+



In [ ]:
df_na = spark.createDataFrame([("A203",None,"Chrome","WIN",33),("A201",'China',None,"MacOS",35),("A205",'UK',"Mozilla","Linux",25)],
                              schema=schema)

In [ ]:
df_na.show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   os|age|
+-------+-------+-------+-----+---+
|   A203|   null| Chrome|  WIN| 33|
|   A201|  China|   null|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [ ]:
df_na.fillna('0').show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   os|age|
+-------+-------+-------+-----+---+
|   A203|      0| Chrome|  WIN| 33|
|   A201|  China|      0|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [ ]:
df_na.fillna({'country':'USA',"browser":"Safari"}).show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   os|age|
+-------+-------+-------+-----+---+
|   A203|    USA| Chrome|  WIN| 33|
|   A201|  China| Safari|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [ ]:
df_na.na.drop().show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   os|age|
+-------+-------+-------+-----+---+
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [6]:
df = spark.read.csv(r'/content/learn-pyspark/chap_2/customer_data.csv',header=True,inferSchema=True)

In [ ]:
df.count()

2000

In [ ]:
df.show(5)

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|Lower class large...|               1|                 3|30-40 years|Family with grown...|     44905|    0|
|Mixed small town ...|               1|                 2|30-40 years|Family with grown...|     37575|    0|
|Mixed small town ...|               1|                 2|30-40 years|Family with grown...|     27915|    0|
|Modern, complete ...|               1|                 3|40-50 years|      Average Family|     19504|    0|
|  Large family farms|               1|                 4|30-40 years|             Farmers|     34943|    0|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
only showing top 5 

In [ ]:
df.printSchema()

root
 |-- Customer_subtype: string (nullable = true)
 |-- Number_of_houses: integer (nullable = true)
 |-- Avg_size_household: integer (nullable = true)
 |-- Avg_age: string (nullable = true)
 |-- Customer_main_type: string (nullable = true)
 |-- Avg_Salary: integer (nullable = true)
 |-- label: integer (nullable = true)



In [ ]:
df.summary().show()

+-------+--------------------+------------------+------------------+-----------+--------------------+-----------------+------------------+
|summary|    Customer_subtype|  Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|       Avg_Salary|             label|
+-------+--------------------+------------------+------------------+-----------+--------------------+-----------------+------------------+
|  count|                2000|              2000|              2000|       2000|                2000|             2000|              2000|
|   mean|                null|            1.1075|            2.6895|       null|                null|     1616908.0835|            0.0605|
| stddev|                null|0.3873225521186316|0.7914562220841646|       null|                null|6822647.757312146|0.2384705099001677|
|    min|Affluent senior a...|                 1|                 1|20-30 years|      Average Family|             1361|                 0|
|    25%|                nu

#Subset of a Dataframe
    Select

    Filter

    Where

In [ ]:
df.select(['Customer_subtype','Avg_Salary']).show(3)

+--------------------+----------+
|    Customer_subtype|Avg_Salary|
+--------------------+----------+
|Lower class large...|     44905|
|Mixed small town ...|     37575|
|Mixed small town ...|     27915|
+--------------------+----------+
only showing top 3 rows



In [ ]:
df.filter(df['Avg_Salary']>100000).count()

181

In [ ]:
df.filter(df['Avg_Salary']>100000).show(3)

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
| High status seniors|               1|                 3|40-50 years|Successful hedonists|   4670288|    0|
|Affluent young fa...|               1|                 3|30-40 years|      Average Family|    762769|    1|
| High status seniors|               1|                 3|50-60 years|Successful hedonists|   9561873|    0|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
only showing top 3 rows



In [ ]:
df.where((df['Avg_Salary'] > 100000) & (df['Number_of_houses']>2)).show(3)

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    596723|    0|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    944444|    0|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    788477|    0|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
only showing top 3 rows



#Aggregations
Any kind of aggregation can be broken simply into three stages, in the following order:

    Split

    Apply
      mean,maxmmin,sum

    Combine

In [ ]:
for col in df.columns:
  if col != 'Avg_Salary':
    print(f"Aggregation for {col}")
    df.groupBy(col).count().orderBy('count',ascending=False).show(truncate=False)

Aggregation for Customer_subtype
+------------------------------------------+-----+
|Customer_subtype                          |count|
+------------------------------------------+-----+
|Lower class large families                |288  |
|Traditional families                      |129  |
|Middle class families                     |122  |
|Large religious families                  |107  |
|Modern, complete families                 |93   |
|Couples with teens 'Married with children'|83   |
|Young and rising                          |78   |
|High status seniors                       |76   |
|Low income catholics                      |72   |
|Mixed seniors                             |71   |
|Village families                          |68   |
|Mixed rurals                              |67   |
|Stable family                             |62   |
|Young all american family                 |62   |
|Young, low educated                       |56   |
|Large family, employed child              |56   

In [ ]:
df.groupBy('Customer_main_type').agg(F.mean('Avg_Salary')).show(3)

+--------------------+------------------+
|  Customer_main_type|   avg(Avg_Salary)|
+--------------------+------------------+
|             Farmers|30209.333333333332|
|       Career Loners|           32272.6|
|Retired and Relig...| 27338.80693069307|
+--------------------+------------------+
only showing top 3 rows



In [ ]:
df.sort("Avg_Salary",ascending=False).show(3)

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
| High status seniors|               1|                 2|60-70 years|Successful hedonists|  48919896|    0|
|High Income, expe...|               1|                 2|50-60 years|Successful hedonists|  48177970|    0|
|High Income, expe...|               1|                 2|50-60 years|Successful hedonists|  48069548|    1|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
only showing top 3 rows



In [ ]:
df.groupBy('Customer_subtype').agg(F.mean('Avg_Salary').alias('mean_salary')).orderBy('mean_salary',ascending=False).show(3)

+--------------------+--------------------+
|    Customer_subtype|         mean_salary|
+--------------------+--------------------+
| High status seniors| 2.507677857894737E7|
|High Income, expe...|2.3839817807692308E7|
|Affluent young fa...|   662068.7777777778|
+--------------------+--------------------+
only showing top 3 rows



Collect
      
    Collect List

    Collect Set

In [ ]:
df.groupby('Customer_subtype').agg(F.collect_set("Number_of_houses")).show(3)

+--------------------+-----------------------------+
|    Customer_subtype|collect_set(Number_of_houses)|
+--------------------+-----------------------------+
|Large family, emp...|                       [1, 2]|
|Religious elderly...|                       [1, 2]|
|Large religious f...|                       [1, 2]|
+--------------------+-----------------------------+
only showing top 3 rows



In [ ]:
df.groupby('Customer_subtype').agg(F.collect_list("Number_of_houses")).show(3)

+--------------------+------------------------------+
|    Customer_subtype|collect_list(Number_of_houses)|
+--------------------+------------------------------+
|Large family, emp...|          [2, 1, 2, 1, 2, 1...|
|Religious elderly...|          [1, 1, 1, 1, 1, 1...|
|Large religious f...|          [2, 1, 1, 2, 1, 1...|
+--------------------+------------------------------+
only showing top 3 rows



In [ ]:
df = df.withColumn('Constant',F.lit('finance'))
df.show(3)

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+--------+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|Constant|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+--------+
|Lower class large...|               1|                 3|30-40 years|Family with grown...|     44905|    0| finance|
|Mixed small town ...|               1|                 2|30-40 years|Family with grown...|     37575|    0| finance|
|Mixed small town ...|               1|                 2|30-40 years|Family with grown...|     27915|    0| finance|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+--------+
only showing top 3 rows



In [ ]:
'''
User-Defined Functions (UDFs)

'''

from pyspark.sql.functions import udf
df.groupby("Avg_age").count().show(3)

+-----------+-----+
|    Avg_age|count|
+-----------+-----+
|70-80 years|    8|
|50-60 years|  373|
|30-40 years|  496|
+-----------+-----+
only showing top 3 rows



In [ ]:
def age_category(age):
    if age  == "20-30 years":
        return "Young"
    elif age== "30-40 years":
        return "Mid Aged"
    elif ((age== "40-50 years") or (age== "50-60 years")) :
        return "Old"
    else:
        return "Very Old"

In [ ]:
age_udf = udf(age_category,StringType())
df=df.withColumn('age_category',age_udf(df['Avg_age']))
df.show(3)

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+--------+------------+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|Constant|age_category|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+--------+------------+
|Lower class large...|               1|                 3|30-40 years|Family with grown...|     44905|    0| finance|    Mid Aged|
|Mixed small town ...|               1|                 2|30-40 years|Family with grown...|     37575|    0| finance|    Mid Aged|
|Mixed small town ...|               1|                 2|30-40 years|Family with grown...|     27915|    0| finance|    Mid Aged|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+--------+------------+
only showing top 3 rows



In [ ]:
min_sal=1361
max_sal=48919896
from pyspark.sql.functions import pandas_udf,PandasUDFType

def  scaled_salary(salary):
    scaled_sal = ((salary-min_sal)/(max_sal-min_sal))
    return scaled_sal

In [ ]:
scaling_udf = pandas_udf(scaled_salary,DoubleType())
df.withColumn("Scaled_salary",scaling_udf(df['Avg_Salary'])).show(3)

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+--------+------------+--------------------+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|Constant|age_category|       Scaled_salary|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+--------+------------+--------------------+
|Lower class large...|               1|                 3|30-40 years|Family with grown...|     44905|    0| finance|    Mid Aged|8.901329526732557E-4|
|Mixed small town ...|               1|                 2|30-40 years|Family with grown...|     37575|    0| finance|    Mid Aged| 7.40291997705982E-4|
|Mixed small town ...|               1|                 2|30-40 years|Family with grown...|     27915|    0| finance|    Mid Aged| 5.42820834679534E-4|
+--------------------+----------------+------------------+-----------+------------------

In [ ]:
'''Joins'''
region_data = spark.createDataFrame([('Family with grown ups','PN'),
                     ('Driven Growers','GJ'),('Conservative families','DD'),('Cruising Seniors','DL'),('Average Family ','MN'),
                     ('Living well','KA'),('Successful hedonists','JH'),('Retired and Religious','AX'),
                     ('Career Loners','HY'),('Farmers','JH')],
                     schema = StructType().add("Customer_main_type","string").add("Region Code","string"))

region_data.show()

+--------------------+-----------+
|  Customer_main_type|Region Code|
+--------------------+-----------+
|Family with grown...|         PN|
|      Driven Growers|         GJ|
|Conservative fami...|         DD|
|    Cruising Seniors|         DL|
|     Average Family |         MN|
|         Living well|         KA|
|Successful hedonists|         JH|
|Retired and Relig...|         AX|
|       Career Loners|         HY|
|             Farmers|         JH|
+--------------------+-----------+



In [ ]:
new_df = df.join(region_data,on='Customer_main_type')
new_df.groupby("Region COde").count().show(3)

+-----------+-----+
|Region COde|count|
+-----------+-----+
|         JH|  287|
|         HY|   15|
|         DD|  236|
+-----------+-----+
only showing top 3 rows



In [ ]:
df.groupBy('Customer_main_type').pivot('Avg_age').sum('Avg_Salary').fillna(0).show(5)

+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+
|  Customer_main_type|20-30 years|30-40 years|40-50 years|50-60 years|60-70 years|70-80 years|
+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+
|             Farmers|          0|     462027|    2031235|     316206|          0|          0|
|       Career Loners|     143998|     176639|      25701|     105193|      32558|          0|
|Retired and Relig...|     126350|     336631|    2975266|    1687711|     335357|      61124|
|Successful hedonists|      42261|  171278764| 1223362814| 1563071675|  200340129|      15518|
|         Living well|     460528|    2965303|    1795405|     331304|          0|          0|
+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+
only showing top 5 rows



#Window Functions


    Aggregations

    Ranking

    Analytics

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col,row_number
win = Window.orderBy(df['Avg_Salary'].desc())
df = df.withColumn('rank', row_number().over(win).alias('rank'))
df.show(3)

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+--------+------------+----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|Constant|age_category|rank|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+--------+------------+----+
| High status seniors|               1|                 2|60-70 years|Successful hedonists|  48919896|    0| finance|    Very Old|   1|
|High Income, expe...|               1|                 2|50-60 years|Successful hedonists|  48177970|    0| finance|         Old|   2|
|High Income, expe...|               1|                 2|50-60 years|Successful hedonists|  48069548|    1| finance|         Old|   3|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+--------+------------+----+
only showing top 3 rows



In [ ]:
win_1 = Window.partitionBy("Customer_subtype").orderBy(df['Avg_Salary'].desc())
df=df.withColumn('rank',row_number().over(win_1))#.alias('rank')

In [ ]:
df.show(5)

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+--------+------------+----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|Constant|age_category|rank|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+--------+------------+----+
|Large family, emp...|               2|                 3|30-40 years|Family with grown...|     49418|    0| finance|    Mid Aged|   1|
|Large family, emp...|               1|                 4|40-50 years|Family with grown...|     48390|    0| finance|         Old|   2|
|Large family, emp...|               1|                 3|40-50 years|Family with grown...|     48272|    0| finance|         Old|   3|
|Large family, emp...|               1|                 2|40-50 years|Family with grown...|     47684|    0| finance|         Old|   4|
|Large family, emp...|               1|         

In [ ]:
df.groupBy('rank').count().orderBy('rank').show(3)

+----+-----+
|rank|count|
+----+-----+
|   1|   39|
|   2|   37|
|   3|   36|
+----+-----+
only showing top 3 rows



In [ ]:
df.filter(col('rank')<4).show(3)

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+--------+------------+----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|Constant|age_category|rank|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+--------+------------+----+
|Large family, emp...|               2|                 3|30-40 years|Family with grown...|     49418|    0| finance|    Mid Aged|   1|
|Large family, emp...|               1|                 4|40-50 years|Family with grown...|     48390|    0| finance|         Old|   2|
|Large family, emp...|               1|                 3|40-50 years|Family with grown...|     48272|    0| finance|         Old|   3|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+--------+------------+----+
only showing top 3 rows



#Spark Structured Streaming

Spark Streaming then, Spark structured streaming now

In [6]:
spark_stream = SparkSession.builder.appName('StructuredStreaming').getOrCreate()

In [45]:
df_1 = spark_stream.createDataFrame([("XN203",'FB',300,30),("XN201",'Twitter',10,19),("XN202",'Insta',500,45)],["user_id","app","time_in_secs","age"]).\
write.csv('/content/csv_folder',mode='append')

In [8]:
schema=StructType().add("user_id","string").add("app","string").add("time_in_secs", "integer").add("age", "integer")

In [10]:
data = spark.readStream.option("sep",",").schema(schema).csv('/content/csv_folder')

In [12]:
data.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- app: string (nullable = true)
 |-- time_in_secs: integer (nullable = true)
 |-- age: integer (nullable = true)



In [13]:
app_count = data.groupBy('app').count()

In [14]:
query = (app_count.writeStream.queryName('count_query').outputMode('complete').format('memory').start())

In [17]:
spark.sql("select * from count_query").toPandas().head(3)

,app,count
0,Insta,1
1,FB,1
2,Twitter,1


In [19]:
fb_data = data.filter(data['app']=='FB')

In [20]:
fb_avg_time = fb_data.groupby('user_id').agg(F.avg('time_in_secs'))

In [21]:
fb_query = (fb_avg_time.writeStream.queryName('fb_query').outputMode('complete').format('memory').start())

In [23]:
spark.sql("select * from fb_query").toPandas().head(3)

,user_id,avg(time_in_secs)
0,XN203,300.0


In [33]:
df_3=spark.createDataFrame([("XN20",'FB',100,30),("XN2044",'FB',10,19),("XN2102",'FB',2000,45)],["user_id","app","time_in_secs","age"]).write.csv("/content/csv_folder",mode='append')

In [34]:
spark.sql("select * from fb_query ").toPandas().head(5)

,user_id,avg(time_in_secs)
0,XN203,150.0
1,XN20,100.0
2,XN201,10.0
3,XN2044,10.0
4,XN202,2000.0


In [35]:
app_df = data.groupBy('app').agg(F.sum('time_in_secs').alias('total_time')).orderBy('total_time',ascending=False)

In [36]:
app_query = (app_df.writeStream.queryName('app_wise_query').outputMode('Complete').format('memory').start())


In [37]:
spark.sql("select * from app_wise_query").toPandas().head(5)

,app,total_time
0,FB,10850
1,Insta,500
2,Twitter,10


In [40]:
df_5=spark.createDataFrame([("XN203",'FB',1000,30),("XN201",'Insta',400,19),("XN202",'Twitter',900,45)],["user_id","app","time_in_secs","age"]).write.csv("csv_folder",mode='append')

In [41]:
spark.sql("select * from app_wise_query").toPandas().head(5)

,app,total_time
0,FB,11350
1,Insta,530
2,Twitter,110


In [42]:
app_df=spark.createDataFrame([('FB','FACEBOOK'),('Insta','INSTAGRAM'),('Twitter','TWITTER')],["app", "full_name"])

In [43]:
app_df.show()

+-------+---------+
|    app|full_name|
+-------+---------+
|     FB| FACEBOOK|
|  Insta|INSTAGRAM|
|Twitter|  TWITTER|
+-------+---------+



In [47]:
app_stream_df = data.join(app_df,'app')
join_query = (app_stream_df.writeStream.queryName('JoinQuery').outputMode('append').format('memory').start())

In [48]:
spark.sql("select * from JoinQuery").toPandas().head(3)

,app,user_id,time_in_secs,age,full_name
0,FB,XN20,100,30,FACEBOOK
1,FB,XN20,100,30,FACEBOOK
2,FB,XN203,300,30,FACEBOOK


#MLLIB

In [10]:
from pyspark.sql import SparkSession

In [11]:
spark = SparkSession.builder.appName('BasicStats').getOrCreate()

In [8]:
!sudo apt install docker-ce

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package docker-ce is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'docker-ce' has no installation candidate


In [12]:
df = spark.read.csv('/content/learn-pyspark/chap_2/customer_data.csv',header=True,inferSchema=True)

In [13]:
df.show(3)

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|Lower class large...|               1|                 3|30-40 years|Family with grown...|     44905|    0|
|Mixed small town ...|               1|                 2|30-40 years|Family with grown...|     37575|    0|
|Mixed small town ...|               1|                 2|30-40 years|Family with grown...|     27915|    0|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
only showing top 3 rows



In [14]:
df.count()

2000

In [21]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Avg_Salary','Avg_size_household'], outputCol='features')

In [22]:
df_n = df.select(['Avg_Salary','Avg_size_household'])

In [23]:
df_n.show(3)

+----------+------------------+
|Avg_Salary|Avg_size_household|
+----------+------------------+
|     44905|                 3|
|     37575|                 2|
|     27915|                 2|
+----------+------------------+
only showing top 3 rows



In [24]:
df_new = assembler.transform(df_n)

In [26]:
df_new.show(3)

+----------+------------------+-------------+
|Avg_Salary|Avg_size_household|     features|
+----------+------------------+-------------+
|     44905|                 3|[44905.0,3.0]|
|     37575|                 2|[37575.0,2.0]|
|     27915|                 2|[27915.0,2.0]|
+----------+------------------+-------------+
only showing top 3 rows



In [27]:
from pyspark.ml.stat import Correlation
pearson_corr = Correlation.corr(df_new,'features')

In [31]:
pearson_corr.show(2,False)

+-----------------------------------------------------------------------------------------+
|pearson(features)                                                                        |
+-----------------------------------------------------------------------------------------+
|1.0                   -0.06585562673466173  
-0.06585562673466173  1.0                   |
+-----------------------------------------------------------------------------------------+



In [32]:
spearman_corr = Correlation.corr(df_new,'features','spearman')
spearman_corr.show(2,False)

+---------------------------------------------------------------------------------------------+
|spearman(features)                                                                           |
+---------------------------------------------------------------------------------------------+
|1.0                    -0.016275912045928723  
-0.016275912045928723  1.0                    |
+---------------------------------------------------------------------------------------------+



In [34]:
'''
Chi Square Test
'''

df=spark.read.csv('chi_sq.csv',inferSchema=True,header=True)#smoker vs non smoker
from pyspark.ml.feature import StringIndexer
marital_indexer = StringIndexer(inputCol="marital", outputCol="marital_num").fit(df)
df = marital_indexer.transform(df)
from pyspark.ml.feature import OneHotEncoder
marital_encoder = OneHotEncoder(inputCol="marital_num", outputCol="marital_vector")
df = marital_encoder.transform(df)
housing_indexer = StringIndexer(inputCol="housing", outputCol="housing_num").fit(df)
df = housing_indexer.transform(df)
housing_encoder = OneHotEncoder(inputCol="housing_num", outputCol="housing_vector")
df = housing_encoder.transform(df)
df_assembler = VectorAssembler(inputCols=['marital_vector','housing_vector'], outputCol="features")
df = df_assembler.transform(df)
chi_sq = ChiSquareTest.test(df, "features", "label").head()

In [39]:
df.show(3)

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|Lower class large...|               1|                 3|30-40 years|Family with grown...|     44905|    0|
|Mixed small town ...|               1|                 2|30-40 years|Family with grown...|     37575|    0|
|Mixed small town ...|               1|                 2|30-40 years|Family with grown...|     27915|    0|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
only showing top 3 rows



In [40]:
#Transformations

#Binarizer
from pyspark.ml.feature import Binarizer
binarizer = Binarizer(threshold=2,inputCol='Number_of_houses',outputCol='binarized_Number_of_houses')
new_df = binarizer.transform(df)
new_df.show(3)

#PCA
from pyspark.ml.feature import PCA
assembler = VectorAssembler(inputCols=[col for col in df.columns if col != 'label'],outputCol="features")
df_new = assembler.transform(df)
pca = PCA(k=2,inputCol="features",outputCol="pca_features")
pca_model = pca.fit(df_new)
pca_comp = pca_model.transform(df_new).select('pca_features')
pca_comp.show(truncate=False)

#Normalizer
from pysparl.ml.featrure import Normalizer
normalizer = Normalizer(inputCol="features",outputCol="norm_features",p=1.0)
normalized_l1_data = normalizer.transform(df_new)
normalized_l1_data.select('norm_features').show(truncate=False)

#Standard Scaling
[In]: from pyspark.ml.feature import StandardScaler
[In]: scaler = StandardScaler(inputCol="features", outputCol="scaled_features",
                             withStd=False, withMean=True)
[In]: scaler_model = scaler.fit(df_new)
[In]: scaled_data = scaler_model.transform(df_new)
[In]: scaled_data.select('scaled_features').show(truncate=False)

#Min-Max Scaling
[In]: from pyspark.ml.feature import MinMaxScaler
[In]: mm_scaler = MinMaxScaler(inputCol="features", outputCol="mm_scaled_features")
[In]: mm_scaler_model = mm_scaler.fit(df_new)
[In]: rescaled_df = mm_scaler_model.transform(df_new)
[In]: rescaled_df.select("features", "mm_scaled_features").show()

#MaxAbsScaler
[In]: from pyspark.ml.feature import MaxAbsScaler
[In]: mxabs_scaler = MaxAbsScaler(inputCol="features", outputCol="mxabs_features")
[In]: mxabs_scaler_model = mxabs_scaler.fit(df_new)
[In]: rescaled_df = mxabs_scaler_model.transform(df_new)
[In]: rescaled_df.select("features", "mxabs_features").show()

#Binning
[In]: from pyspark.ml.feature import Bucketizer
[In]: splits = [0.0,1.0,2.0,3.0,4.0,5.0,float("inf")]
[In]: bucketizer = Bucketizer(splits=splits, inputCol="label", outputCol="label_bins")
[In]: binned_df = bucketizer.transform(df)
[In]: binned_df.select(['label','label_bins']).show(10,False)
[In]: binned_df.groupBy('label_bins').count().show()
print(bucketizer.getSplits())-1

#Classification Model

In [41]:
#Step 1: Load the Dataset
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Classification').getOrCreate()
df = spark.read.csv('/content/learn-pyspark/chap_5/classification_data.csv',inferSchema=True,
                    header=True)

In [42]:
df.count()

46751

In [43]:
df.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- loan_purpose: string (nullable = true)
 |-- is_first_loan: integer (nullable = true)
 |-- total_credit_card_limit: integer (nullable = true)
 |-- avg_percentage_credit_card_limit_used_last_year: double (nullable = true)
 |-- saving_amount: integer (nullable = true)
 |-- checking_amount: integer (nullable = true)
 |-- is_employed: integer (nullable = true)
 |-- yearly_salary: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- dependent_number: integer (nullable = true)
 |-- label: integer (nullable = true)



In [44]:
#Step 2: Explore the Dataframe
df.show(3)

+-------+------------+-------------+-----------------------+-----------------------------------------------+-------------+---------------+-----------+-------------+---+----------------+-----+
|loan_id|loan_purpose|is_first_loan|total_credit_card_limit|avg_percentage_credit_card_limit_used_last_year|saving_amount|checking_amount|is_employed|yearly_salary|age|dependent_number|label|
+-------+------------+-------------+-----------------------+-----------------------------------------------+-------------+---------------+-----------+-------------+---+----------------+-----+
|    A_1|    personal|            1|                   7900|                                            0.8|         1103|           6393|          1|        16400| 42|               4|    0|
|    A_2|    personal|            0|                   3300|                                           0.29|         2588|            832|          1|        75500| 56|               1|    0|
|    A_3|    personal|            0|    

In [48]:
df.groupBy('label').count().alias('label_group').show()

+-----+-----+
|label|count|
+-----+-----+
|    1|16201|
|    0|30550|
+-----+-----+



In [49]:
df.groupBy('loan_purpose').count().show()

+------------+-----+
|loan_purpose|count|
+------------+-----+
|      others| 6763|
|   emergency| 7562|
|    property|11388|
|  operations|10580|
|    personal|10458|
+------------+-----+



In [50]:
#Step 3: Data Transformation

from pyspark.ml.feature import OneHotEncoder, StringIndexer,VectorAssembler

loan_purpose_indexer = StringIndexer(inputCol='loan_purpose',outputCol='loan_purpose_index').fit(df)
df = loan_purpose_indexer.transform(df)

In [53]:
loan_encoder = OneHotEncoder(inputCol="loan_purpose_index",outputCol="loan_purpose_vector").fit(df)
df = loan_encoder.transform(df)

In [54]:
df.show(3)

+-------+------------+-------------+-----------------------+-----------------------------------------------+-------------+---------------+-----------+-------------+---+----------------+-----+------------------+-------------------+
|loan_id|loan_purpose|is_first_loan|total_credit_card_limit|avg_percentage_credit_card_limit_used_last_year|saving_amount|checking_amount|is_employed|yearly_salary|age|dependent_number|label|loan_purpose_index|loan_purpose_vector|
+-------+------------+-------------+-----------------------+-----------------------------------------------+-------------+---------------+-----------+-------------+---+----------------+-----+------------------+-------------------+
|    A_1|    personal|            1|                   7900|                                            0.8|         1103|           6393|          1|        16400| 42|               4|    0|               2.0|      (4,[2],[1.0])|
|    A_2|    personal|            0|                   3300|                

In [57]:
df_vec_assembler = VectorAssembler(inputCols=['is_first_loan',
 'total_credit_card_limit',
 'avg_percentage_credit_card_limit_used_last_year',
 'saving_amount',
 'checking_amount',
 'is_employed',
 'yearly_salary',
 'age',
 'dependent_number',
 'loan_purpose_vector'], outputCol="features")

In [58]:
df = df_vec_assembler.transform(df)
df.select(['features','label']).show(10,False)

+--------------------------------------------------------------------+-----+
|features                                                            |label|
+--------------------------------------------------------------------+-----+
|[1.0,7900.0,0.8,1103.0,6393.0,1.0,16400.0,42.0,4.0,0.0,0.0,1.0,0.0] |0    |
|[0.0,3300.0,0.29,2588.0,832.0,1.0,75500.0,56.0,1.0,0.0,0.0,1.0,0.0] |0    |
|[0.0,7600.0,0.9,1651.0,8868.0,1.0,59000.0,46.0,1.0,0.0,0.0,1.0,0.0] |0    |
|[1.0,3400.0,0.38,1269.0,6863.0,1.0,26000.0,55.0,8.0,0.0,0.0,1.0,0.0]|0    |
|[0.0,2600.0,0.89,1310.0,3423.0,1.0,9700.0,41.0,4.0,0.0,0.0,0.0,1.0] |1    |
|[0.0,7600.0,0.51,1040.0,2406.0,1.0,22900.0,52.0,0.0,0.0,1.0,0.0,0.0]|0    |
|[1.0,6900.0,0.82,2408.0,5556.0,1.0,34800.0,48.0,4.0,0.0,1.0,0.0,0.0]|0    |
|[0.0,5700.0,0.56,1933.0,4139.0,1.0,32500.0,64.0,2.0,0.0,0.0,1.0,0.0]|0    |
|[1.0,3400.0,0.95,3866.0,4131.0,1.0,13300.0,23.0,3.0,0.0,0.0,1.0,0.0]|0    |
|[0.0,2900.0,0.91,88.0,2725.0,1.0,21100.0,52.0,1.0,0.0,0.0,1.0,0.0]  |1    |

In [59]:
model_df = df.select(['features','label'])

In [60]:
#Step 4: Splitting into Train and Test Data
train_df,test_df = model_df.randomSplit([0.70,0.30])

In [61]:
from pyspark.ml.classification import LogisticRegression
log_reg = LogisticRegression().fit(train_df)
lr_summary = log_reg.summary
lr_summary.accuracy

0.8933528122717312

In [62]:
lr_summary.areaUnderROC

0.9590400974642718

In [63]:
print(lr_summary.precisionByLabel)

[0.9229937473555545, 0.838929650410013]


In [64]:
print(lr_summary.recallByLabel)

[0.9132052653611796, 0.855771770714097]


In [65]:
predictions = log_reg.transform(test_df)

In [66]:
predictions.show(10) 

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(13,[0,1,2,3,4,7]...|    1|[-6.8525548483602...|[0.00105563685221...|       1.0|
|(13,[0,1,2,3,4,7]...|    0|[1.06037808660394...|[0.74276279151388...|       0.0|
|(13,[0,1,2,3,4,7]...|    1|[-2.2167418772478...|[0.09825710497559...|       1.0|
|(13,[0,1,2,3,4,7]...|    0|[6.69302569032580...|[0.99876200785470...|       0.0|
|(13,[0,1,2,3,4,7]...|    1|[1.26395396565278...|[0.77970600995358...|       0.0|
|(13,[0,1,2,3,4,7,...|    1|[-6.5102931218942...|[0.00148583249764...|       1.0|
|(13,[0,1,2,3,4,7,...|    1|[-5.6831282363765...|[0.00339135627548...|       1.0|
|(13,[0,1,2,3,4,7,...|    1|[-5.0823029621792...|[0.00616732929554...|       1.0|
|(13,[0,1,2,3,4,7,...|    1|[-6.3285962618067...|[0.00178135813495...|       1.0|
|(13,[0,1,2,3,4,

In [ ]:
[In]: model_predictions = log_reg.transform(test_df)
[In]: model_predictions = log_reg.evaluate(test_df)
[In]: model_predictions.accuracy
[Out]: 0.8945984906300347
[In]: model_predictions.areaUnderROC
[Out]: 0.9594316478468224
[In]: print(model_predictions.recallByLabel)
[Out]: [0.9129581151832461, 0.8608235010835541]
[In]: print(model_predictions.precisionByLabel)
[Out]: [0.9234741162452006, 0.8431603773584906] 

In [69]:
#Step 6: Hyperparameter Tuning
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier()
rf_model = rf.fit(train_df)

In [70]:
model_predictions = rf_model.transform(test_df)

In [73]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()
rf = RandomForestClassifier()
paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [5,10,20,25,30])
             .addGrid(rf.maxBins, [20,30,40 ])
             .addGrid(rf.numTrees, [5, 20,50])
             .build())
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
cv_model = cv.fit(train_df)


In [74]:
best_rf_model = cv_model.bestModel
model_predictions = best_rf_model.transform(test_df)
model_predictions = best_rf_model.transform(test_df)
true_pos=model_predictions.filter(model_predictions['label']==1).filter(model_predictions['prediction']==1).count()
actual_pos=model_predictions.filter(model_predictions['label']==1).count()
pred_pos=model_predictions.filter(model_predictions['prediction']==1).count()
recall_rate=float(true_pos)/(actual_pos)
print(recall_rate)

0.9035920726672172


#Regression

Linear Regression

In [76]:
#Step 1: Create the Spark Session Object

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Supervised').getOrCreate()
df=spark.read.csv('/content/learn-pyspark/chap_6/Linear_regression_dataset.csv',inferSchema=True,header=True)
df.show(3)

+-----+-----+-----+-----+-----+-----+
|var_1|var_2|var_3|var_4|var_5|label|
+-----+-----+-----+-----+-----+-----+
|  734|  688|   81|0.328|0.259|0.418|
|  700|  600|   94| 0.32|0.247|0.389|
|  712|  705|   93|0.311|0.247|0.417|
+-----+-----+-----+-----+-----+-----+
only showing top 3 rows



In [77]:
df.columns

['var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'label']

In [80]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
vec_assembler = VectorAssembler(inputCols=['var_1', 'var_2', 'var_3', 'var_4', 'var_5'],outputCol='features')

In [82]:
features_df = vec_assembler.transform(df)

In [84]:
features_df.show(3)

+-----+-----+-----+-----+-----+-----+--------------------+
|var_1|var_2|var_3|var_4|var_5|label|            features|
+-----+-----+-----+-----+-----+-----+--------------------+
|  734|  688|   81|0.328|0.259|0.418|[734.0,688.0,81.0...|
|  700|  600|   94| 0.32|0.247|0.389|[700.0,600.0,94.0...|
|  712|  705|   93|0.311|0.247|0.417|[712.0,705.0,93.0...|
+-----+-----+-----+-----+-----+-----+--------------------+
only showing top 3 rows



In [85]:
model_df = features_df.select(['features','label'])

In [89]:
train,test = model_df.randomSplit([.80,.20])

In [90]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression()

In [91]:
lr_model = lr.fit(train)
predictions = lr_model.transform(test)
predictions.show(3)

+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[501.0,774.0,51.0...|0.315|0.32941576721355015|
|[524.0,665.0,65.0...|0.336|0.33526828055190494|
|[527.0,569.0,75.0...|0.341|0.33404522272680504|
+--------------------+-----+-------------------+
only showing top 3 rows



In [94]:
model_evaluate = lr_model.evaluate(test)
print(model_evaluate.r2)
print(model_evaluate.meanSquaredError)

0.8665399365021725
0.00014751436252392696


In [98]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import GeneralizedLinearRegression
glr = GeneralizedLinearRegression()


In [99]:
glr_model = glr.fit(train)
glr_model.summary

Coefficients:
    Feature Estimate Std Error T Value P Value
(Intercept)   0.1817    0.0158 11.5292  0.0000
      var_1   0.0003    0.0000 23.8724  0.0000
      var_2   0.0001    0.0000  5.2442  0.0000
      var_3   0.0002    0.0001  2.4339  0.0151
      var_4  -0.6537    0.0672 -9.7327  0.0000
      var_5   0.5017    0.0597  8.4053  0.0000

(Dispersion parameter for gaussian family taken to be 0.0001)
    Null deviance: 1.0777 on 969 degrees of freedom
Residual deviance: 0.1404 on 969 degrees of freedom
AIC: -5843.7712

In [103]:
model_prediction = glr_model.evaluate(test)
model_prediction.predictions.show()

+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[501.0,774.0,51.0...|0.315|0.32941576721355015|
|[524.0,665.0,65.0...|0.336|0.33526828055190494|
|[527.0,569.0,75.0...|0.341|0.33404522272680504|
|[537.0,660.0,63.0...|0.326|0.32878148867545504|
|[550.0,631.0,76.0...|0.318| 0.3376626624781536|
|[558.0,740.0,60.0...| 0.36|0.34866277732553574|
|[564.0,648.0,74.0...|0.337|0.35119608054214957|
|[567.0,587.0,84.0...|0.349|0.34744221279909987|
|[568.0,708.0,57.0...|0.347| 0.3464119284930939|
|[569.0,776.0,53.0...|0.348| 0.3590516826571347|
|[570.0,655.0,66.0...| 0.34|0.34562368436206414|
|[570.0,662.0,73.0...|0.337| 0.3488342303101433|
|[573.0,634.0,75.0...|0.342|0.34848477303445524|
|[574.0,586.0,81.0...| 0.36|0.35112672117515176|
|[575.0,680.0,68.0...|0.344| 0.3538959828401379|
|[575.0,864.0,55.0...|0.379| 0.3650182165612126|
|[579.0,497.0,91.0...|0.352|0.33939234105473914|
|[583.0,794.0,55.0..

In [ ]:

[In]: glr = GeneralizedLinearRegression(family='Binomial')
[In]: glr_model = glr.fit(train)
[In]: model_predictions=glr_model.evaluate(test)
[In]: model_predictions.aic
[Out]: 336.991
[In]: glr = GeneralizedLinearRegression(family='Poisson')
[In]: glr_model = glr.fit(train)
[In]: predictions=glr_model.evaluate(test)
[In]: predictions.aic
[Out]: 266.53
[In]: glr = GeneralizedLinearRegression(family='Gamma')
[In]: glr_model = glr.fit(train)
[In]:model_predictions=glr_model.evaluate(test)
[In]: model_predictions.aic
[Out]: -1903.81


In [104]:
from pyspark.ml.regression import DecisionTreeRegressor
d_tree = DecisionTreeRegressor()
d_tree_model = d_tree.fit(train)
model_pred = d_tree_model.transform(test)
from pyspark.ml.evaluation import RegressionEvaluator
dt_evaluator = RegressionEvaluator(metricName="r2")
dt_r2 = dt_evaluator.evaluate(model_pred)
print(f'The r-square value of DecisionTreeRegressor is {dt_r2}')
dt_evaluator = RegressionEvaluator(metricName="rmse")
dt_r3 = dt_evaluator.evaluate(model_pred)
print(f'The MSE value of DecisionTreeRegressor is {dt_r3}')

The r-square value of DecisionTreeRegressor is 0.8105437747921171
The MSE value of DecisionTreeRegressor is 0.014470912910963866


In [ ]:

[In]: from pyspark.ml.regression import RandomForestRegressor
[In]: rf = RandomForestRegressor()
[In]: rf_model = rf.fit(train)
[In]:  rf_model.featureImportances
[Out]: SparseVector(5, {0: 0.4395, 1: 0.045, 2: 0.0243, 3: 0.2725, 4: 0.2188})
[In]: rf_model.getNumTrees
[Out]: 20
[In]: model_predictions = rf_model.transform(test)
[In]: model_predictions.show()
[Out]:
[In]:rf_evaluator = RegressionEvaluator(metricName='r2')
[In]: rf_r2 = rf_evaluator.evaluate(model_predictions)
[In]: print(f'The r-square value of RandomForestRegressor is {rf_r2}')
[Out]: The r-square value of RandomForestRegressor is 0.8215863293044671
[In]: rf_evaluator = RegressionEvaluator(metricName='rmse')
[In]: rf_rmse = rf_evaluator.evaluate(model_predictions)
[In]: print(f'The rmse value of RandomForestRegressor is {rf_rmse}')
[Out]: The rmse value of RandomForestRegressor is 0.01365275410722947


In [ ]:
[In]: from pyspark.ml.regression import GBTRegressor
[In]: gbt = GBTRegressor()
[In]: gbt_model=gbt.fit(train)
[In]: gbt_model.featureImportances
[Out]: SparseVector(5, {0: 0.2325, 1: 0.2011, 2: 0.1645, 3: 0.2268, 4: 0.1751})
[In]: model_predictions = gbt_model.transform(test)
[In]: model_predictions.show()

In [ ]:

[In]: gbt_evaluator = RegressionEvaluator(metricName='r2')
[In]: gbt_r2 = gbt_evaluator.evaluate(model_predictions)
[In]: print(f'The r-square value of GradientBoostedRegressor is {gbt_r2}')
[Out]: The r-square value of GradientBoostedRegressor is 0.8477273892307596
[In]: gbt_evaluator = RegressionEvaluator(metricName='rmse')
[In]: gbt_rmse = gbt_evaluator.evaluate(model_predictions)
[In]: print(f'The rmse value of GradientBoostedRegressor is {gbt_rmse}')
[Out]: The rmse value of GradientBoostedRegressor is 0.013305445803592103


#Building Multiple Models for Binary Classification Tasks

In [4]:
spark = SparkSession.builder.appName('Classification').getOrCreate()

In [4]:
df=spark.read.csv('/content/learn-pyspark/chap_6/bank_data.csv',inferSchema=True,header=True)

In [10]:
df.count()

41188

In [5]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- emp.var.rate: double (nullable = true)
 |-- cons.price.idx: double (nullable = true)
 |-- cons.conf.idx: double (nullable = true)
 |-- euribor3m: double (nullable = true)
 |-- nr.employed: double (nullable = true)
 |-- target_class: string (nullable = true)



In [9]:
df.groupBy('target_class').count().show()

+------------+-----+
|target_class|count|
+------------+-----+
|          no|36548|
|         yes| 4640|
+------------+-----+



In [7]:
from pyspark.sql import functions as F
from pyspark.sql import *
df = df.withColumn('label', F.when(df.target_class == 'no',F.lit(0)).otherwise(F.lit(1)))

In [8]:
df.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1| 4640|
|    0|36548|
+-----+-----+



In [11]:
df.show(3)

+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+------------+-----+
|age|      job|marital|  education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|target_class|label|
+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+------------+-----+
| 56|housemaid|married|   basic.4y|     no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0|          no|    0|
| 57| services|married|high.school|unknown|     no|  no|telephone|  may|        mon|     149|       1|  999|       0|nonexistent|         1.1|      

In [12]:
columnList = [item[0] for item in df.dtypes if item[1].startswith('string')]
print(columnList)

['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome', 'target_class']


In [14]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
for col in columnList:
  stringIndexer = StringIndexer(inputCol=col,outputCol = col+"_index")
  indexer = stringIndexer.fit(df).transform(df)
  OHE = OneHotEncoder(inputCol=col+"_index",outputCol=col+"_OHE")
  df = OHE.fit(indexer).transform(indexer)
df.show(3)

+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+------------+-----+---------+--------------+-------------+-------------+---------------+-------------+-------------+-------------+-------------+-------------+----------+-------------+-------------+-----------+-----------+-------------+-----------------+---------------+--------------+-------------+------------------+----------------+
|age|      job|marital|  education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|target_class|label|job_index|       job_OHE|marital_index|  marital_OHE|education_index|education_OHE|default_index|  default_OHE|housing_index|  housing_OHE|loan_index|     loan_OHE|contact_index|contact_OHE|month_index|    month_OHE|day_of_week_index|day_of_week_OHE|poutcome_i

In [19]:
vec_assembler = VectorAssembler(inputCols=['age','job_OHE','marital_OHE','education_OHE','default_OHE','housing_OHE','loan_OHE','contact_OHE','month_OHE','day_of_week_OHE','poutcome_OHE'],
                                outputCol='features').transform(df)

In [20]:
model_df = vec_assembler.select(['features','label'])

In [21]:
train,test = model_df.randomSplit([0.80,0.20])

In [22]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression()
lr_model = lr.fit(train)
print( lr_model.coefficients)

[0.005722582008627255,0.15727819558514727,-0.14201202760878004,0.01714461405939384,-0.10329218403593356,-0.01428419125299655,0.6440537276177551,0.029192018830323792,0.02627766258907318,0.027447525918169197,0.13981178229791694,0.8259729708954047,-0.17824183779906333,-0.04374697540005107,-0.32496797080210205,0.019570289253074552,-0.07239630574646001,-0.20780005139753924,-0.08310530034373678,-0.07111826315281648,0.03281280902860767,0.12504756827541225,2.0778453542266937,1.5047821345747283,-0.0075844790831816515,0.027171650324812613,0.03920847522135311,-0.0063674354161558635,0.9324290213426493,-1.5745184403207328,-1.5169288509269674,-1.6209402068154237,-0.8619351201882424,-1.6875874872891972,-0.8148595133259126,-0.08090259944744509,-0.23482716808861206,0.3258386614482381,0.045133896689987164,-0.15764459557703878,0.14558419290776223,0.05030179223684514,-2.2701144529281265,-2.2210323072315776]


In [ ]:

[In]: lr_summary=lr_model.summary
[In]: lr_summary.accuracy
[Out]: 0.673079623648364
[In]: lr_summary.areaUnderROC
[Out]: 0.7186044694483512
[In]: lr_summary.weightedRecall
[Out]: 0.673079623648364
[In]: lr_summary.weightedPrecision
[Out]: 0.6750967624018298


In [24]:
lr_model.summary.precisionByThreshold.show()

+------------------+------------------+
|         threshold|         precision|
+------------------+------------------+
|0.9118768770791803|0.8620689655172413|
|0.9024399707564577|0.8620689655172413|
|0.8844543665985334|0.8409090909090909|
|0.8617592335124054|0.8050847457627118|
|0.8493386986015156|0.7891156462585034|
|0.8376719742351242|0.7542857142857143|
|0.8302654023270768|0.7378640776699029|
|0.8220723823193792|0.7457627118644068|
|0.8104220305544716|0.7386363636363636|
|0.7921248221087368|0.7406143344709898|
|0.7743560032993397|0.7445482866043613|
|0.7426850179346008|0.7378917378917379|
|0.7285806016424878|0.7368421052631579|
|0.7183914042091777|0.7439024390243902|
|0.7078835780165118|0.7431818181818182|
|0.6966839411308392| 0.744136460554371|
|0.6855726427077629|0.7354709418837675|
| 0.675817374012148|0.7381404174573055|
|0.6631235312824618| 0.737410071942446|
|0.6472661074834476|0.7316239316239316|
+------------------+------------------+
only showing top 20 rows



In [25]:
model_predictions = lr_model.transform(test)
model_predictions.columns


['features', 'label', 'rawPrediction', 'probability', 'prediction']

In [26]:
model_predictions.select(['label','probability','prediction']).show(10,False)

+-----+----------------------------------------+----------+
|label|probability                             |prediction|
+-----+----------------------------------------+----------+
|0    |[0.8846318498897413,0.1153681501102587] |0.0       |
|0    |[0.8846318498897413,0.1153681501102587] |0.0       |
|0    |[0.5018360593101189,0.4981639406898811] |0.0       |
|0    |[0.9093786422228192,0.09062135777718083]|0.0       |
|0    |[0.9093786422228192,0.09062135777718083]|0.0       |
|0    |[0.9079538848993545,0.09204611510064553]|0.0       |
|0    |[0.9074745101446231,0.0925254898553769] |0.0       |
|0    |[0.9065090303158331,0.09349096968416692]|0.0       |
|0    |[0.9030583630397901,0.09694163696020985]|0.0       |
|0    |[0.8963475266290245,0.1036524733709755] |0.0       |
+-----+----------------------------------------+----------+
only showing top 10 rows



In [27]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
lr_evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
lr_auroc = lr_evaluator.evaluate(model_predictions)
print(f'The auroc value of Logistic Regression Model is {lr_auroc}')

The auroc value of Logistic Regression Model is 0.7664365083565627


In [28]:
lr_evaluator = BinaryClassificationEvaluator(metricName='areaUnderPR')
lr_aupr = lr_evaluator.evaluate(model_predictions)
print(f'The aupr value of Logistic Regression Model is {lr_aupr}')

The aupr value of Logistic Regression Model is 0.40137308136205163


In [ ]:
In]: true_pos=model_predictions.filter(model_predictions['label']==1).filter(model_predictions['prediction']==1).count()
[In]: actual_pos=model_predictions.filter(model_predictions['label']==1).count()
[In]: pred_pos=model_predictions.filter(model_predictions['prediction']==1).count()
Recall
[In]: float(true_pos)/(actual_pos)
[Out]: 0.6701030927835051
Precision
[In]: float(true_pos)/(pred_pos)
[Out]: 0.6478405315614618

In [30]:
from pyspark.ml.classification import DecisionTreeClassifier

In [31]:
dt = DecisionTreeClassifier()
dt_model = dt.fit(train)
model_predictions = dt_model.transform(test)

In [32]:
model_predictions.select(['label','probability','prediction']).show(10,False)

+-----+----------------------------------------+----------+
|label|probability                             |prediction|
+-----+----------------------------------------+----------+
|0    |[0.8737654320987654,0.12623456790123458]|0.0       |
|0    |[0.8737654320987654,0.12623456790123458]|0.0       |
|0    |[0.5449101796407185,0.4550898203592814] |0.0       |
|0    |[0.8892603172673677,0.11073968273263235]|0.0       |
|0    |[0.8892603172673677,0.11073968273263235]|0.0       |
|0    |[0.8892603172673677,0.11073968273263235]|0.0       |
|0    |[0.8892603172673677,0.11073968273263235]|0.0       |
|0    |[0.8892603172673677,0.11073968273263235]|0.0       |
|0    |[0.8892603172673677,0.11073968273263235]|0.0       |
|0    |[0.8892603172673677,0.11073968273263235]|0.0       |
+-----+----------------------------------------+----------+
only showing top 10 rows



In [ ]:

[In]: dt_evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
[In]: dt_auroc = dt_evaluator.evaluate(model_predictions)
[In]: print(f'The auc value of Decision Tree Classifier Model is {dt_auroc}')
[Out]: The auc value of Decision Tree Classifier Model is 0.516199386190993
[In]: dt_evaluator = BinaryClassificationEvaluator(metricName='areaUnderPR')
[In]: dt_aupr = dt_evaluator.evaluate(model_predictions)
[In]: print(f'The aupr value of Decision Tree Model is {dt_aupr}')
[Out]: The aupr value of Decision Tree Model is 0.46771834172588167
[In]: true_pos=model_predictions.filter(model_predictions['label']==1).filter(model_predictions['prediction']==1).count()
[In]: actual_pos=model_predictions.filter(model_predictions['label']==1).count()
[In]: pred_pos=model_predictions.filter(model_predictions['prediction']==1).count()
[In]: float(true_pos)/(actual_pos)
[Out]: 0.6907216494845361
[In]: float(true_pos)/(pred_pos)
[Out]: 0.6661143330571665


In [ ]:
Step 1: Build and Train SVM Model
[In]: from pyspark.ml.classification import LinearSVC
[In]: lsvc = LinearSVC()
[In]: lsvc_model = lsvc.fit(train)
[In]: model_predictions = lsvc_model.transform(test)
[In]: model_predictions.columns
[Out]: ['features', 'label', 'rawPrediction', 'prediction']
[In]:model_predictions.select(['label','prediction']).show(10,False)
[Out]:
Step 2: Evaluate Performance on Test Data
[In]: svc_evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
[In]: svc_auroc = svc_evaluator.evaluate(model_predictions)
[In]: print(f'The auc value of SupportVectorClassifier  is {svc_auroc}')
[Out]: The auc value of SupportVectorClassifier  is 0.7043772749366973
[In]: svc_evaluator = BinaryClassificationEvaluator(metricName='areaUnderPR')
[In]: svc_aupr =svc_evaluator.evaluate(model_predictions)
[In]: print(f'The aupr value of SupportVectorClassifier Model is {svc_aupr}')
[Out]: The aupr value of SupportVectorClassifier Model is 0.6567277377856992
[In]: true_pos=model_predictions.filter(model_predictions['label']==1).filter(model_predictions['prediction']==1).count()
[In]: actual_pos=model_predictions.filter(model_predictions['label']==1).count()
[In]: pred_pos=model_predictions.filter(model_predictions['prediction']==1).count()
[In]: float(true_pos)/(actual_pos)
[Out]: 0.7774914089347079
[In]: float(true_pos)/(pred_pos)
[Out]: 0.600132625994695

In [ ]:
Step 1: Build and Train SVM Model
[In]: from pyspark.ml.classification import NaiveBayes
[In]: nb = NaiveBayes()
[In]: nb_model = nb.fit(train)
[In]: model_predictions = nb_model.transform(test)
[In]: model_predictions.select(['label','probability','prediction']).show(10,False)
1h 7m remaining
© Pramod Singh 2019
P. SinghLearn PySparkhttps://doi.org/10.1007/978-1-4842-4961-1_6
6. Supervised Machine Learning
Pramod Singh1 
(1)
Bangalore, Karnataka, India
 

Machine learning can be broadly divided into four categories: supervised machine learning and unsupervised machine learning and, to a lesser extent, semi-supervised machine learning and reinforcement machine learning. Because supervised machine learning drives a lot of business applications and significantly affects our day-to-day lives, it is considered one of the most important categories.

This chapter reviews supervised machine learning, using multiple algorithms. In Chapter 7, we’ll look at unsupervised machine learning. I’ll begin by providing an overview of the different categories of supervised machine learning. In the second section, I will cover various regression methods, and we will build machine learning models, using PySpark’s MLlib library. The third and final section of this chapter focuses on classification, using multiple machine learning algorithms.
Supervised Machine Learning Primer
In supervised machine learning, as the name suggests, the learning process is supervised, as the machine learning algorithm being used corrects its predictions, based on the actual output. In supervised machine learning, the correct labels or output is already known during the model training phase, and, hence, the error can be reduced accordingly. In short, we try to map the relationship between the input data and output label in such a way as to pick up the signals from the training data and generalize about the unseen data as well. The training of the model consists of comparing the actual output with the predicted output and then making the changes in predictions, to reduce the total error between what is actual and what is predicted. The supervised machine learning process followed is as shown in Figure 6-1.
../images/478390_1_En_6_Chapter/478390_1_En_6_Fig1_HTML.jpg
Figure 6-1

Supervised learning approach
The data used for training the model is preprocessed, and features are created accordingly. Once the machine learning model is trained, it can be used to make predictions on the unseen data. So, in the preceding figure, we can see how the model is trained, using input data and how now, the trained model is used to predict whether the new transaction is genuine. This type of learning is predominantly used in cases in which historical data is available and predictions must be made on future data. The further categorization of supervised learning is based on types of output or target variables being used for prediction:

    Regression

    Classification

Regression is used when the target value that is being predicted is continuous or numerical in nature. For example, predicting salary based on a given number of years of experience or education falls under the category of regression.
Note

Although there are multiple types of regression, in this chapter, I’ll focus on linear regression and some of its associated algorithms, as you’ll see shortly.
Classification is used if the target variable is a discrete value or categorical in nature. For example, predicting whether a customer will churn out is a type of classification problem, as shown in Figure 6-2.
../images/478390_1_En_6_Chapter/478390_1_En_6_Fig2_HTML.jpg
Figure 6-2

Types of suprvised tasks
Classification tasks can further be broken down into two categories: binary class and multi-class, as shown in Figure 6-3.
../images/478390_1_En_6_Chapter/478390_1_En_6_Fig3_HTML.png
Figure 6-3

Types of classes
Binary Classification
When the target or output variable contains only up to two categories, it is referred to as binary classification. So, every record in the data can only fall under one of the two groups. For example:

    Yes or no

    Group A or group B

    Sell or not sell

    Positive or negative

    Accepted or rejected

Multi-class Classification
When the target or output variable contains more than two categories, it is referred to as multi-class classification. So, there can be multiple groups within the data, and every record can belong to any of the groups. For example:

    Yes or no or maybe

    Group A or group B or group C

    Category 1 or category 2 or category 3 or others

    Rank 1 or rank 2 or rank 3 or rank 4 or rank 5

Another useful property of supervised learning is that the model’s performance can be evaluated on training and test data. Based on the type of model (classification or regression), the evaluation metric can be applied, and performance results can be measured. In this chapter, I will cover how to build machine learning models to execute regression and binary classification.
Building a Linear Regression Model
Linear regression refers to modeling the relationship between a set of independent variables and the output or dependent (numerical) variables. If the input variables include more than one variable, this is known as multivariable linear regression. In short, it is assumed that the dependent variable is a linear combination of other independent variables.
$$ \overline{y}={B}_0+{B}_1\ast {X}_1+{B}_2\ast {X}_{2+\dots } $$
Here X1, X2, … are the independent variables that are used to predict the output variable. The output of the linear regression is a straight line, which minimizes the actual vs. predicted values. A linear regression model cannot handle nonlinear data, as it’s only possible to model linearly separable data, therefore polynomial regression is used for nonlinear data, and the output is generally a curve, instead of a straight line, as shown in Figure 6-4.
../images/478390_1_En_6_Chapter/478390_1_En_6_Fig4_HTML.jpg
Figure 6-4

Types of regression

Linear regression also assumes that data is normally distributed, in order to improve prediction. Linear regression is one of the ways to predict continuous values, and you will see now how we can use other alternatives, to predict numerical output.

The following sections focus on solving regression tasks, using multiple machine learning algorithms. I will begin with data ingestion and exploratory data analysis and then build models. The steps 1 to 4 will be the same for all the regression models.
Note

Complete datasets, along with the relevant code, are available for reference from the GitHub repository for this book and execute best on Spark 2.3 and higher versions.

Let’s build a linear regression model, using Spark’s MLlib library, and predict the target variable, using the input features.
Reviewing the Data Information

The dataset that we are going to use for this example is a sample dataset and contains a total of 1,232 rows and 6 columns. We have to use 5 input variables to predict the target variable, using the linear regression model.
Step 1: Create the Spark Session Object
We start the Jupyter notebook, import SparkSession, and create a new SparkSession object to use with Spark.
[In]: from pyspark.sql import SparkSession
[In]: spark=SparkSession.builder.appName('supervised_ml').getOrCreate()
Step 2: Read the Dataset
We then load and read the dataset within Spark, using DataFrame. We have to make sure that we have opened PySpark from the same directory folder where the dataset is available, or else we have to mention the directory path of the data folder.
[In]: df=spark.read.csv('Linear_regression_dataset.csv',inferSchema=True,header=True)
[In]:print((df.count(), len(df.columns)))
[Out]: (1232, 6)
The preceding output confirms the size of our dataset, so we can then validate the datatypes of the input values, to check if we have to change/cast any column datatypes. In this example, all columns contain integer or double values that are already aligned with our requirements.
[In]: df.printSchema()
[Out]: root
 |-- var_1: integer (nullable = true)
 |-- var_2: integer (nullable = true)
 |-- var_3: integer (nullable = true)
 |-- var_4: double (nullable = true)
 |-- var_5: double (nullable = true)
 |-- output: double (nullable = true)
There is a total of six columns, of which five are input columns (var_1 to var_5) and target columns (label). We can now use the describe function to go over statistical measures of the dataset.
[In]: df.show(10)
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figa_HTML.jpg
Step 3: Feature Engineering
This is the part where we create a single vector combining all input features, by using Spark’s VectorAssembler. It creates only a single feature that captures the input values for that particular row. So, instead of five input columns, the engine essentially translates the features into a single column with five input values, in the form of a list.
[In]: from pyspark.ml.linalg import Vector
[In]: from pyspark.ml.feature import VectorAssembler
We will pass all five input columns, to create a single features column.
[In]: df.columns
[Out]: ['var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'label']
[In]: vec_assmebler=VectorAssembler(inputCols=['var_1', 'var_2', 'var_3', 'var_4', 'var_5'],outputCol='features')
[In]: features_df=vec_assmebler.transform(df)
[In]: features_df.printSchema()
[Out]: root
 |-- var_1: integer (nullable = true)
 |-- var_2: integer (nullable = true)
 |-- var_3: integer (nullable = true)
 |-- var_4: double (nullable = true)
 |-- var_5: double (nullable = true)
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
As you can see, we have an additional column (features), which contains the single dense vector for all of the inputs. We then take a subset of the dataframe and select only the features column and the label column, to build the linear regression model.
[In]: df.select(['features','label']).show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figb_HTML.jpg
Step 4: Split the Dataset
Let’s split the dataset into training and test datasets, in order to train and evaluate the performance of the linear regression model. We split it according to a 70/30 ratio and train our model on 70% of the dataset. We can print the shape of the train and test data, to validate the size.
[In]: train, test = df.randomSplit([0.75, 0.25])
[In]:print(f"Size of train Dataset : {train.count()}" )
[Out]: Size of train Dataset : 911
[In]: print(f"Size of test Dataset : {test.count()}" )
[Out]:  Size of test Dataset : 321
Step 5: Build and Train Linear Regression Model
Now we build and train the linear regression model, using features, input, and label columns. We first import the linear regression from MLlib, as follows:
[In]: from pyspark.ml.regression import LinearRegression
[In]: lr = LinearRegression()
Note

For simplicity, all the machine learning models built in this chapter use default hyperparameters. Readers can use their own set of hyperparameters.
[In]:lr_model = lr.fit(train)
[In]: predictions_df=lr_model.transform(test)
[In]: predictions_df.show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figc_HTML.jpg
Step 6: Evaluate Linear Regression Model on Test Data
To check the performance of the model on unseen or test data, we make use of evaluate.
[In]: model_predictions=lr_model.evaluate(test)
[In]: model_predictions.r2
[Out]: 0.8855561089304634
[In]: print(model_predictions.meanSquaredError)
[Out]:0.00013305453514672318
Generalized Linear Model Regression
The generalized linear model (GLM) is an advanced version of linear regression that considers the target variable to have an error distribution other than a preferred normal distribution. The GLM generalizes linear regression, using a link function, so that variance is a function of the predicted value itself. Let’s try to build the GLM on the same dataset and see if it performs better than a simple linear regression model. First, we must import the GLM from MLlib.
[In]: from pyspark.ml.regression import GeneralizedLinearRegression
Step 1: Build and Train Generalized Linear Regression Model
[In]: glr = GeneralizedLinearRegression()
[In]: glr_model = glr.fit(train)
[In]: glr_model.coefficients
[Out]: DenseVector([0.0003, 0.0001, 0.0001, -0.6374, 0.4822])
We can get the coefficient values, using coefficient functions of that model. Here we can see that one of the features has a negative coefficient value. We can get more information about the GLM model, by using the summary function. It returns all the details, such as coefficient value, std error, AIC (Akaike information criterion) value, and p value.
[In]: glr_model.summary
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figd_HTML.jpg
Step 2: Evaluate the Model Performance on Test Data
[In]: model_predictions=glr_model.evaluate(test)
[In]: model_predictions.predictions.show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Fige_HTML.jpg
The Akaike information criterion (AIC) is an evaluation parameter of relative performance of quality of models for the same set dataset. AIC is mainly used to select among multiple models for a given dataset. A lesser value of AIC indicates that the model is of good quality. AIC tries to strike a balance between the variance and bias of the model. Therefore, it deals with the chances both of overfitting and underfitting. The model with the lowest AIC score is preferred over other models.
[In]: model_predictions.aic
[Out]: -1939.88
We can run the GLM for multiple distributions, such as

    1.

    Binomial
     
    2.

    Poisson
     
    3.

    Gamma
     
    4.

    Tweedie
     

[In]: glr = GeneralizedLinearRegression(family='Binomial')
[In]: glr_model = glr.fit(train)
[In]: model_predictions=glr_model.evaluate(test)
[In]: model_predictions.aic
[Out]: 336.991
[In]: glr = GeneralizedLinearRegression(family='Poisson')
[In]: glr_model = glr.fit(train)
[In]: predictions=glr_model.evaluate(test)
[In]: predictions.aic
[Out]: 266.53
[In]: glr = GeneralizedLinearRegression(family='Gamma')
[In]: glr_model = glr.fit(train)
[In]:model_predictions=glr_model.evaluate(test)
[In]: model_predictions.aic
[Out]: -1903.81

Here we can see that our default GLM model with Gaussian distribution has the lowest AIC value, compared to others.
Decision Tree Regression

The decision tree regression algorithm can be used for both regression and classification. It is quite powerful in terms of fitting the data well but comes with the high risk of sometimes overfitting the data. Decision trees contain multiple splits based on entropy or Gini indexes. The deeper the tree, the higher the chance of overfitting the data. In our example, we will build a decision tree for predicting the target value, with the default value of parameters (maxdepth = 5).
Step 1: Build and Train Decision Tree Regressor Model
[In]: from pyspark.ml.regression import DecisionTreeRegressor
[In]: dec_tree = DecisionTreeRegressor()
[In]: dec_tree_model = dec_tree.fit(train)
[In]: dec_tree_model.featureImportances
[Out]: SparseVector(5, {0: 0.9641, 1: 0.0193, 2: 0.0029, 3: 0.0053, 4: 0.0084})
Step 2: Evaluate the Model Performance on Test Data
[In]: model_predictions = dec_tree_model.transform(test)
[In]: model_predictions.show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figf_HTML.jpg
We import RegressionEvaluation from MLlib, to evaluate the performance of the decision tree on test data. As of now, there are two metrics available for evaluation: r2 and RMSE (root mean squared error). r2 mainly suggests how much of the variation in the dataset can be attributed to regression. Therefore, the higher the r2, the better the performance of the model. On the other hand, RMSE suggests the total errors the model is making, in terms of the difference between actual and predicted values.
[In]: from pyspark.ml.evaluation import RegressionEvaluator
[In]: dt_evaluator = RegressionEvaluator(metricName='r2')
[In]: dt_r2 = dt_evaluator.evaluate(model_predictions)
[In]: print(f'The r-square value of DecisionTreeRegressor is {dt_r2}')
[Out]: The r-square value of DecisionTreeRegressor is 0.8093834699203476
[In]: dt_evaluator = RegressionEvaluator(metricName='rmse')
[In]: dt_rmse = dt_evaluator.evaluate(model_predictions)
[In]: print(f'The rmse value of DecisionTreeRegressor is {dt_rmse}')
[Out]: The rmse value of DecisionTreeRegressor is 0.014111932287681688

The r2 value of this particular model is close to 0.81, which is a little lower than that of a simple linear regression model.
Random Forest Regressors

Random forest regressors are a collection of multiple individual decision trees built using different samples of data. The whole idea of combining these individual trees is to take majority voting or averages (in case of regression) to generalize effectively. A random forest is, therefore, an ensembling technique that takes a bagging approach. It can be used for regression as well as for classification tasks. Because decision trees tend to overfit the data, random forests remove the element of high variance, by taking the means of the predicted values from individual trees. In our example, we will build a random forest model for regression, using default parameters (numTrees = 20)
Step 1: Build and Train Random Forest Regressor Model
[In]: from pyspark.ml.regression import RandomForestRegressor
[In]: rf = RandomForestRegressor()
[In]: rf_model = rf.fit(train)
[In]:  rf_model.featureImportances
[Out]: SparseVector(5, {0: 0.4395, 1: 0.045, 2: 0.0243, 3: 0.2725, 4: 0.2188})
As you can see, the number of trees in the random forest is equal to 20. This number can be increased.
[In]: rf_model.getNumTrees
[Out]: 20
[In]: model_predictions = rf_model.transform(test)
[In]: model_predictions.show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figg_HTML.jpg
Step 2: Evaluate the Model Performance on Test Data
We can again use r2and RMSE as the evaluation parameter of the random forest model.
[In]:rf_evaluator = RegressionEvaluator(metricName='r2')
[In]: rf_r2 = rf_evaluator.evaluate(model_predictions)
[In]: print(f'The r-square value of RandomForestRegressor is {rf_r2}')
[Out]: The r-square value of RandomForestRegressor is 0.8215863293044671
[In]: rf_evaluator = RegressionEvaluator(metricName='rmse')
[In]: rf_rmse = rf_evaluator.evaluate(model_predictions)
[In]: print(f'The rmse value of RandomForestRegressor is {rf_rmse}')
[Out]: The rmse value of RandomForestRegressor is 0.01365275410722947

As you can see, it clearly outperforms the decision tree regressor and has a higher r2. The performance of this model can further be enhanced with hyperparameter tuning.
Gradient-Boosted Tree Regressor

A gradient-boosted tree (GBT) regressor is also an ensembling technique, which uses boosting under the hood. Boosting refers to making use of individual weak learners in order to boost the performance of the overall model. One major difference between bagging and boosting is that in bagging, the individual models that are built are parallel in nature, meaning they can be built independent of each other, but in boosting, the individual models are built in a sequential manner. In a gradient boosting approach, the second model focuses on the errors made by the first model and tries to reduce overall errors for those data points. Similarly, the next model tries to reduce the errors made by the previous model. In this way, the overall error of prediction is reduced. In the following example, we will build a GBT regressor with default parameters.
Step 1: Build and Train a GBT Regressor Model
[In]: from pyspark.ml.regression import GBTRegressor
[In]: gbt = GBTRegressor()
[In]: gbt_model=gbt.fit(train)
[In]: gbt_model.featureImportances
[Out]: SparseVector(5, {0: 0.2325, 1: 0.2011, 2: 0.1645, 3: 0.2268, 4: 0.1751})
[In]: model_predictions = gbt_model.transform(test)
[In]: model_predictions.show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figh_HTML.jpg
Step 2: Evaluate the Model Performance on Test Data
[In]: gbt_evaluator = RegressionEvaluator(metricName='r2')
[In]: gbt_r2 = gbt_evaluator.evaluate(model_predictions)
[In]: print(f'The r-square value of GradientBoostedRegressor is {gbt_r2}')
[Out]: The r-square value of GradientBoostedRegressor is 0.8477273892307596
[In]: gbt_evaluator = RegressionEvaluator(metricName='rmse')
[In]: gbt_rmse = gbt_evaluator.evaluate(model_predictions)
[In]: print(f'The rmse value of GradientBoostedRegressor is {gbt_rmse}')
[Out]: The rmse value of GradientBoostedRegressor is 0.013305445803592103

As you can see, the GBT regressor outperforms the random forestmodel. With r2 being close to 0.85, it can be considered the final model, after proper tuning.
Building Multiple Models for Binary Classification Tasks

In this third and final section of the chapter, you will see how to build multiple machine learning models for binary classification tasks. The data that we are going to use for this is a subset of an open source Bank Marketing Data Set from the UCI ML repository, available at https://archive.ics.uci.edu/ml/datasets/Bank+Marketing.

There are two reasons for selecting only a subset of this data. The first is to maintain the class balance for the classification task, so as not to make it an anomalous detection category task. Another reason for selecting only a subset of the features is to limit the amount of signals in the data, as some of the features in the dataset strongly affect the output and, therefore, are ignored in this exercise.

The dataset contains 9,500 rows and 8 columns. The idea is to predict if the user will subscribe to another product or service (term deposit), based on the other attributes, such as age, job, loan, etc. This is a typical requirement in which machine learning is leveraged to find the top users who can be targeted by the business for cross-selling or upselling.

I’ll begin with the logistic regression model.
Logistic Regression

Logistic regression is considered to be one of baseline models, owing to its simplicity and interpretability. Under the hood, it is quite similar to linear regression. It also assumes that output is a linear combination of the dependent variables, but to keep the output between 0 and 1, as it returns the probability as output, it makes use of a nonlinear function (sigmoid), which produces an S curve instead of a straight line (linear regression).

We’ll start by building the baseline in Steps 1–3 and then complete the logistic regression model with default hyperparameters, in Steps 4–6.
Step 1: Read the Dataset
[In]: df=spark.read.csv('bank_data.csv',inferSchema=True,header=True)
[In]: df.count()
[Out]: 9501
[In]: df.columns
[Out]: ['age', 'job', 'marital', 'education', 'default', 'housing','loan', 'target_class']
[In]: df.printSchema()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figi_HTML.jpg
As you can see, the input columns are all the columns, except for the target class column. The target class is also well-balanced, in terms of the count of yes and no labels. We will have to convert yeses and noes into 1s and 0s, as well as rename the target_class column to “label,” which is the default acceptance column name in machine learning model parameters.
[In]: df.groupBy('target_class').count().show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figj_HTML.jpg
Step 2: Feature Engineering for Model
[In]: from pyspark.sql import functions as F
[In]: from pyspark.sql import ∗
[In]: df=df.withColumn("label", F.when(df.target_class =='no', F.lit(0)).otherwise(F.lit(1)))
[In]: df.groupBy('label').count().show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figk_HTML.jpg
Now that we have renamed the output column “label” and converted the target class to 1s and 0s, the next step is to create features for the model. Because we have categorical columns, such as job and edu, we will have to use StringIndexer and OneHotEncoder to convert them into a numerical format. We create a Python function, cat_to_num, to convert all the categorical features into numerical ones.
[In]: from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
[In]: def cat_to_num(df):
    for col in df.columns:
        stringIndexer = StringIndexer(inputCol=col, outputCol=col+"_index")
        model = stringIndexer.fit(df)
        indexed = model.transform(df)
        encoder = OneHotEncoder(inputCol=col+"_index", outputCol=col+"_vec")
        df = encoder.transform(indexed)
    df_assembler = VectorAssembler(inputCols=['age','marital_vec','education_vec','default_vec','housing_vec','loan_vec'], outputCol="features")
    df = df_assembler.transform(df)
    return df.select(['features','label'])
We just select the new features column and target label column, as we don’t need the earlier original columns for model training.
[In]: df_new=cat_to_num(df)
[In]: df_new.show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figl_HTML.jpg

Now we have all the input features merged into a single dense vector ('features'), along with output column labels, which we can use to train the machine learning models. The new dataframe created using only two columns (features, label) is now called df_new and will be used for every model. We can now split this new dataframe into train and test datasets. We can split the data into a 75%/25% ratio, using the randomplit function.
Step 3: Split the Data into Train and Test Datasets
[In]: train, test = df_new.randomSplit([0.75, 0.25])
[In]: print(f"Size of train Dataset : {train.count()}" )
[Out]: 7121
[In]: print(f"Size of test Dataset : {test.count()}" )
[Out]: 2380
Step 4: Build and Train the Logistic Regression Model
[In]: from pyspark.ml.classification import LogisticRegression
[In]: lr = LogisticRegression()
[In]: lr_model = lr.fit(train)
[In]:print( lr_model.coefficients)
[Out]:
[0.0272019114172,-0.647672064875,0.229030508111,-0.77074788287,-12.36869511,-12.8865599132,-13.2257790609,-12.6705131313,-13.0023164274,-13.0747662586,-12.6985757761,1.42220523957,0.301582233094,-0.0127231892838,0.218471149577,0.332362933568]

Once the model is built, we can make use of the internal function summary, which offers important details regarding the model, such as ROC curve, precision, recall, AUC (area under the curve), etc.
Step 5: Evaluate Performance on Training Data
[In]: lr_summary=lr_model.summary
[In]: lr_summary.accuracy
[Out]: 0.673079623648364
[In]: lr_summary.areaUnderROC
[Out]: 0.7186044694483512
[In]: lr_summary.weightedRecall
[Out]: 0.673079623648364
[In]: lr_summary.weightedPrecision
[Out]: 0.6750967624018298
Here, using the summary function , we can view the model’s performance on train data, such as its accuracy, AUC, weighted recall, and precision. We can also view additional details—such as how precision varies for various threshold values, the relation between precision and recall, and how recall varies with different threshold values—to pick the right threshold value for the model. These also can be plotted, to view the relationships.
[In]: lr_summary.precisionByThreshold.show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figm_HTML.jpg
[In]: lr_summary.roc.show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Fign_HTML.jpg
[In]: lr_summary.recallByThreshold.show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figo_HTML.jpg
[In]: lr_summary.pr.show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figp_HTML.jpg
Step 6: Evaluate Performance on Test Data
[In]: model_predictions = lr_model.transform(test)
[In]: model_predictions.columns
[Out]: ['features', 'label', 'rawPrediction', 'probability', 'prediction']
[In]: model_predictions.select(['label','probability','prediction']).show(10,False)
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figq_HTML.jpg
As you can see, the prediction column shows the model prediction for each of the records in the test data. The probability column shows the values for both classes (0 & 1). The probability at 0th index is of 0; the other is for a prediction of 1. The evaluation of the logistic regression model on test data can be done using BinaryClassEvaluator . We can get the area under ROC and that under the PR curve, as shown following:
[In]:from pyspark.ml.evaluation import BinaryClassificationEvaluator
[In]:lr_evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
[In]: lr_auroc = lr_evaluator.evaluate(model_predictions)
[In]: print(f'The auroc value of Logistic Regression Model is {lr_auroc}')
[Out]: The auroc value of Logistic Regression Model is 0.7092938229110143
[In]: lr_evaluator = BinaryClassificationEvaluator(metricName='areaUnderPR')
[In]: lr_aupr = lr_evaluator.evaluate(model_predictions)
[In]: print(f'The aupr value of Logistic Regression Model is {lr_aupr}')
[Out]: The aupr value of Logistic Regression Model is 0.6630743130940658
[In]: true_pos=model_predictions.filter(model_predictions['label']==1).filter(model_predictions['prediction']==1).count()
[In]: actual_pos=model_predictions.filter(model_predictions['label']==1).count()
[In]: pred_pos=model_predictions.filter(model_predictions['prediction']==1).count()
Recall
[In]: float(true_pos)/(actual_pos)
[Out]: 0.6701030927835051
Precision
[In]: float(true_pos)/(pred_pos)
[Out]: 0.6478405315614618
Decision Tree Classifier

As mentioned earlier, decision trees can be used for classification as well as regression. Here, we will build a decision tree with default hyperparameters and use it to predict whether the user will opt for the new term deposit plan.
Step 1: Build and Train Decision Tree Classifier Model
[In]: from pyspark.ml.classification import DecisionTreeClassifier
[In]: dt = DecisionTreeClassifier()
[In]: dt_model = dt.fit(train)
[In]: model_predictions = dt_model.transform(test)
[Out]: model_predictions.select(['label','probability','prediction']).show(10,False)
../images/478390_1_En_6_Chapter/478390_1_En_6_Figr_HTML.jpg
Step 2: Evaluate Performance on Test Data
[In]: dt_evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
[In]: dt_auroc = dt_evaluator.evaluate(model_predictions)
[In]: print(f'The auc value of Decision Tree Classifier Model is {dt_auroc}')
[Out]: The auc value of Decision Tree Classifier Model is 0.516199386190993
[In]: dt_evaluator = BinaryClassificationEvaluator(metricName='areaUnderPR')
[In]: dt_aupr = dt_evaluator.evaluate(model_predictions)
[In]: print(f'The aupr value of Decision Tree Model is {dt_aupr}')
[Out]: The aupr value of Decision Tree Model is 0.46771834172588167
[In]: true_pos=model_predictions.filter(model_predictions['label']==1).filter(model_predictions['prediction']==1).count()
[In]: actual_pos=model_predictions.filter(model_predictions['label']==1).count()
[In]: pred_pos=model_predictions.filter(model_predictions['prediction']==1).count()
[In]: float(true_pos)/(actual_pos)
[Out]: 0.6907216494845361
[In]: float(true_pos)/(pred_pos)
[Out]: 0.6661143330571665
Support Vector Machines Classifiers
Support vector machines (SVMs) are used for classification tasks, as they find the hyperplane that maximizes the margin (perpendicular distance) between two classes. All the instances and target classes are represented as vectors in high-dimensional space, and the SVM finds the closest two points from the two classes that support the best separating line or hyperplane, as shown in Figure 6-5.
../images/478390_1_En_6_Chapter/478390_1_En_6_Fig5_HTML.jpg
Figure 6-5

Support vector machine

For nonlinearly separable data, there are different kernel tricks to separate the classes. In our example, we will build a linearly separable support vector classifier with default hyperparameters.
Step 1: Build and Train SVM Model
[In]: from pyspark.ml.classification import LinearSVC
[In]: lsvc = LinearSVC()
[In]: lsvc_model = lsvc.fit(train)
[In]: model_predictions = lsvc_model.transform(test)
[In]: model_predictions.columns
[Out]: ['features', 'label', 'rawPrediction', 'prediction']
[In]:model_predictions.select(['label','prediction']).show(10,False)
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figs_HTML.jpg
Step 2: Evaluate Performance on Test Data
[In]: svc_evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
[In]: svc_auroc = svc_evaluator.evaluate(model_predictions)
[In]: print(f'The auc value of SupportVectorClassifier  is {svc_auroc}')
[Out]: The auc value of SupportVectorClassifier  is 0.7043772749366973
[In]: svc_evaluator = BinaryClassificationEvaluator(metricName='areaUnderPR')
[In]: svc_aupr =svc_evaluator.evaluate(model_predictions)
[In]: print(f'The aupr value of SupportVectorClassifier Model is {svc_aupr}')
[Out]: The aupr value of SupportVectorClassifier Model is 0.6567277377856992
[In]: true_pos=model_predictions.filter(model_predictions['label']==1).filter(model_predictions['prediction']==1).count()
[In]: actual_pos=model_predictions.filter(model_predictions['label']==1).count()
[In]: pred_pos=model_predictions.filter(model_predictions['prediction']==1).count()
[In]: float(true_pos)/(actual_pos)
[Out]: 0.7774914089347079
[In]: float(true_pos)/(pred_pos)
[Out]: 0.600132625994695
Naive Bayes Classifier

Naive Bayes (NB) classifiers work on the principle of conditional probability and assume absolute independence between predictors. An NB classifier doesn’t have many hyperparameters and can outperform some of the most sophisticated algorithms out there. In the following example, we will build an NB classifier and evaluate its performance on the test data.
Step 1: Build and Train SVM Model
[In]: from pyspark.ml.classification import NaiveBayes
[In]: nb = NaiveBayes()
[In]: nb_model = nb.fit(train)
[In]: model_predictions = nb_model.transform(test)
[In]: model_predictions.select(['label','probability','prediction']).show(10,False)
[Out]:

Step 2: Evaluate Performance on Test Data
[In]: nb_evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
[In]: nb_auroc = nb_evaluator.evaluate(model_predictions)
[In]: print(f'The auc value of NB Classifier is {nb_auroc}')
[Out]: The auc value of NB Classifier is 0.43543736717760884
[In]: nb_evaluator = BinaryClassificationEvaluator(metricName='areaUnderPR')
[In]: nb_aupr =nb_evaluator.evaluate(model_predictions)
[In]: print(f'The aupr value of NB Classifier Model is {nb_aupr}')
[Out]: The aupr value of NB Classifier Model is 0.4321001351769349
[In]: true_pos=model_predictions.filter(model_predictions['label']==1).filter(model_predictions['prediction']==1).count()
[In]: actual_pos=model_predictions.filter(model_predictions['label']==1).count()
[In]: pred_pos=model_predictions.filter(model_predictions['prediction']==1).count()
[In]: float(true_pos)/(actual_pos)
[Out]: 0.586
[In]: float(true_pos)/(pred_pos)
[Out]: 0.625


In [ ]:
Step 1: Build and Train the GBT Model
[In]: from pyspark.ml.classification import GBTClassifier
[In]: gbt = GBTClassifier()
[In]: gbt_model = gbt.fit(train)
[In]: model_predictions = gbt_model.transform(test)
[In]: model_predictions.select(['label','probability','prediction']).show(10,False)

1h 5m remaining
© Pramod Singh 2019
P. SinghLearn PySparkhttps://doi.org/10.1007/978-1-4842-4961-1_6
6. Supervised Machine Learning
Pramod Singh1 
(1)
Bangalore, Karnataka, India
 

Machine learning can be broadly divided into four categories: supervised machine learning and unsupervised machine learning and, to a lesser extent, semi-supervised machine learning and reinforcement machine learning. Because supervised machine learning drives a lot of business applications and significantly affects our day-to-day lives, it is considered one of the most important categories.

This chapter reviews supervised machine learning, using multiple algorithms. In Chapter 7, we’ll look at unsupervised machine learning. I’ll begin by providing an overview of the different categories of supervised machine learning. In the second section, I will cover various regression methods, and we will build machine learning models, using PySpark’s MLlib library. The third and final section of this chapter focuses on classification, using multiple machine learning algorithms.
Supervised Machine Learning Primer
In supervised machine learning, as the name suggests, the learning process is supervised, as the machine learning algorithm being used corrects its predictions, based on the actual output. In supervised machine learning, the correct labels or output is already known during the model training phase, and, hence, the error can be reduced accordingly. In short, we try to map the relationship between the input data and output label in such a way as to pick up the signals from the training data and generalize about the unseen data as well. The training of the model consists of comparing the actual output with the predicted output and then making the changes in predictions, to reduce the total error between what is actual and what is predicted. The supervised machine learning process followed is as shown in Figure 6-1.
../images/478390_1_En_6_Chapter/478390_1_En_6_Fig1_HTML.jpg
Figure 6-1

Supervised learning approach
The data used for training the model is preprocessed, and features are created accordingly. Once the machine learning model is trained, it can be used to make predictions on the unseen data. So, in the preceding figure, we can see how the model is trained, using input data and how now, the trained model is used to predict whether the new transaction is genuine. This type of learning is predominantly used in cases in which historical data is available and predictions must be made on future data. The further categorization of supervised learning is based on types of output or target variables being used for prediction:

    Regression

    Classification

Regression is used when the target value that is being predicted is continuous or numerical in nature. For example, predicting salary based on a given number of years of experience or education falls under the category of regression.
Note

Although there are multiple types of regression, in this chapter, I’ll focus on linear regression and some of its associated algorithms, as you’ll see shortly.
Classification is used if the target variable is a discrete value or categorical in nature. For example, predicting whether a customer will churn out is a type of classification problem, as shown in Figure 6-2.
../images/478390_1_En_6_Chapter/478390_1_En_6_Fig2_HTML.jpg
Figure 6-2

Types of suprvised tasks
Classification tasks can further be broken down into two categories: binary class and multi-class, as shown in Figure 6-3.
../images/478390_1_En_6_Chapter/478390_1_En_6_Fig3_HTML.png
Figure 6-3

Types of classes
Binary Classification
When the target or output variable contains only up to two categories, it is referred to as binary classification. So, every record in the data can only fall under one of the two groups. For example:

    Yes or no

    Group A or group B

    Sell or not sell

    Positive or negative

    Accepted or rejected

Multi-class Classification
When the target or output variable contains more than two categories, it is referred to as multi-class classification. So, there can be multiple groups within the data, and every record can belong to any of the groups. For example:

    Yes or no or maybe

    Group A or group B or group C

    Category 1 or category 2 or category 3 or others

    Rank 1 or rank 2 or rank 3 or rank 4 or rank 5

Another useful property of supervised learning is that the model’s performance can be evaluated on training and test data. Based on the type of model (classification or regression), the evaluation metric can be applied, and performance results can be measured. In this chapter, I will cover how to build machine learning models to execute regression and binary classification.
Building a Linear Regression Model
Linear regression refers to modeling the relationship between a set of independent variables and the output or dependent (numerical) variables. If the input variables include more than one variable, this is known as multivariable linear regression. In short, it is assumed that the dependent variable is a linear combination of other independent variables.
$$ \overline{y}={B}_0+{B}_1\ast {X}_1+{B}_2\ast {X}_{2+\dots } $$
Here X1, X2, … are the independent variables that are used to predict the output variable. The output of the linear regression is a straight line, which minimizes the actual vs. predicted values. A linear regression model cannot handle nonlinear data, as it’s only possible to model linearly separable data, therefore polynomial regression is used for nonlinear data, and the output is generally a curve, instead of a straight line, as shown in Figure 6-4.
../images/478390_1_En_6_Chapter/478390_1_En_6_Fig4_HTML.jpg
Figure 6-4

Types of regression

Linear regression also assumes that data is normally distributed, in order to improve prediction. Linear regression is one of the ways to predict continuous values, and you will see now how we can use other alternatives, to predict numerical output.

The following sections focus on solving regression tasks, using multiple machine learning algorithms. I will begin with data ingestion and exploratory data analysis and then build models. The steps 1 to 4 will be the same for all the regression models.
Note

Complete datasets, along with the relevant code, are available for reference from the GitHub repository for this book and execute best on Spark 2.3 and higher versions.

Let’s build a linear regression model, using Spark’s MLlib library, and predict the target variable, using the input features.
Reviewing the Data Information

The dataset that we are going to use for this example is a sample dataset and contains a total of 1,232 rows and 6 columns. We have to use 5 input variables to predict the target variable, using the linear regression model.
Step 1: Create the Spark Session Object
We start the Jupyter notebook, import SparkSession, and create a new SparkSession object to use with Spark.
[In]: from pyspark.sql import SparkSession
[In]: spark=SparkSession.builder.appName('supervised_ml').getOrCreate()
Step 2: Read the Dataset
We then load and read the dataset within Spark, using DataFrame. We have to make sure that we have opened PySpark from the same directory folder where the dataset is available, or else we have to mention the directory path of the data folder.
[In]: df=spark.read.csv('Linear_regression_dataset.csv',inferSchema=True,header=True)
[In]:print((df.count(), len(df.columns)))
[Out]: (1232, 6)
The preceding output confirms the size of our dataset, so we can then validate the datatypes of the input values, to check if we have to change/cast any column datatypes. In this example, all columns contain integer or double values that are already aligned with our requirements.
[In]: df.printSchema()
[Out]: root
 |-- var_1: integer (nullable = true)
 |-- var_2: integer (nullable = true)
 |-- var_3: integer (nullable = true)
 |-- var_4: double (nullable = true)
 |-- var_5: double (nullable = true)
 |-- output: double (nullable = true)
There is a total of six columns, of which five are input columns (var_1 to var_5) and target columns (label). We can now use the describe function to go over statistical measures of the dataset.
[In]: df.show(10)
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figa_HTML.jpg
Step 3: Feature Engineering
This is the part where we create a single vector combining all input features, by using Spark’s VectorAssembler. It creates only a single feature that captures the input values for that particular row. So, instead of five input columns, the engine essentially translates the features into a single column with five input values, in the form of a list.
[In]: from pyspark.ml.linalg import Vector
[In]: from pyspark.ml.feature import VectorAssembler
We will pass all five input columns, to create a single features column.
[In]: df.columns
[Out]: ['var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'label']
[In]: vec_assmebler=VectorAssembler(inputCols=['var_1', 'var_2', 'var_3', 'var_4', 'var_5'],outputCol='features')
[In]: features_df=vec_assmebler.transform(df)
[In]: features_df.printSchema()
[Out]: root
 |-- var_1: integer (nullable = true)
 |-- var_2: integer (nullable = true)
 |-- var_3: integer (nullable = true)
 |-- var_4: double (nullable = true)
 |-- var_5: double (nullable = true)
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
As you can see, we have an additional column (features), which contains the single dense vector for all of the inputs. We then take a subset of the dataframe and select only the features column and the label column, to build the linear regression model.
[In]: df.select(['features','label']).show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figb_HTML.jpg
Step 4: Split the Dataset
Let’s split the dataset into training and test datasets, in order to train and evaluate the performance of the linear regression model. We split it according to a 70/30 ratio and train our model on 70% of the dataset. We can print the shape of the train and test data, to validate the size.
[In]: train, test = df.randomSplit([0.75, 0.25])
[In]:print(f"Size of train Dataset : {train.count()}" )
[Out]: Size of train Dataset : 911
[In]: print(f"Size of test Dataset : {test.count()}" )
[Out]:  Size of test Dataset : 321
Step 5: Build and Train Linear Regression Model
Now we build and train the linear regression model, using features, input, and label columns. We first import the linear regression from MLlib, as follows:
[In]: from pyspark.ml.regression import LinearRegression
[In]: lr = LinearRegression()
Note

For simplicity, all the machine learning models built in this chapter use default hyperparameters. Readers can use their own set of hyperparameters.
[In]:lr_model = lr.fit(train)
[In]: predictions_df=lr_model.transform(test)
[In]: predictions_df.show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figc_HTML.jpg
Step 6: Evaluate Linear Regression Model on Test Data
To check the performance of the model on unseen or test data, we make use of evaluate.
[In]: model_predictions=lr_model.evaluate(test)
[In]: model_predictions.r2
[Out]: 0.8855561089304634
[In]: print(model_predictions.meanSquaredError)
[Out]:0.00013305453514672318
Generalized Linear Model Regression
The generalized linear model (GLM) is an advanced version of linear regression that considers the target variable to have an error distribution other than a preferred normal distribution. The GLM generalizes linear regression, using a link function, so that variance is a function of the predicted value itself. Let’s try to build the GLM on the same dataset and see if it performs better than a simple linear regression model. First, we must import the GLM from MLlib.
[In]: from pyspark.ml.regression import GeneralizedLinearRegression
Step 1: Build and Train Generalized Linear Regression Model
[In]: glr = GeneralizedLinearRegression()
[In]: glr_model = glr.fit(train)
[In]: glr_model.coefficients
[Out]: DenseVector([0.0003, 0.0001, 0.0001, -0.6374, 0.4822])
We can get the coefficient values, using coefficient functions of that model. Here we can see that one of the features has a negative coefficient value. We can get more information about the GLM model, by using the summary function. It returns all the details, such as coefficient value, std error, AIC (Akaike information criterion) value, and p value.
[In]: glr_model.summary
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figd_HTML.jpg
Step 2: Evaluate the Model Performance on Test Data
[In]: model_predictions=glr_model.evaluate(test)
[In]: model_predictions.predictions.show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Fige_HTML.jpg
The Akaike information criterion (AIC) is an evaluation parameter of relative performance of quality of models for the same set dataset. AIC is mainly used to select among multiple models for a given dataset. A lesser value of AIC indicates that the model is of good quality. AIC tries to strike a balance between the variance and bias of the model. Therefore, it deals with the chances both of overfitting and underfitting. The model with the lowest AIC score is preferred over other models.
[In]: model_predictions.aic
[Out]: -1939.88
We can run the GLM for multiple distributions, such as

    1.

    Binomial
     
    2.

    Poisson
     
    3.

    Gamma
     
    4.

    Tweedie
     

[In]: glr = GeneralizedLinearRegression(family='Binomial')
[In]: glr_model = glr.fit(train)
[In]: model_predictions=glr_model.evaluate(test)
[In]: model_predictions.aic
[Out]: 336.991
[In]: glr = GeneralizedLinearRegression(family='Poisson')
[In]: glr_model = glr.fit(train)
[In]: predictions=glr_model.evaluate(test)
[In]: predictions.aic
[Out]: 266.53
[In]: glr = GeneralizedLinearRegression(family='Gamma')
[In]: glr_model = glr.fit(train)
[In]:model_predictions=glr_model.evaluate(test)
[In]: model_predictions.aic
[Out]: -1903.81

Here we can see that our default GLM model with Gaussian distribution has the lowest AIC value, compared to others.
Decision Tree Regression

The decision tree regression algorithm can be used for both regression and classification. It is quite powerful in terms of fitting the data well but comes with the high risk of sometimes overfitting the data. Decision trees contain multiple splits based on entropy or Gini indexes. The deeper the tree, the higher the chance of overfitting the data. In our example, we will build a decision tree for predicting the target value, with the default value of parameters (maxdepth = 5).
Step 1: Build and Train Decision Tree Regressor Model
[In]: from pyspark.ml.regression import DecisionTreeRegressor
[In]: dec_tree = DecisionTreeRegressor()
[In]: dec_tree_model = dec_tree.fit(train)
[In]: dec_tree_model.featureImportances
[Out]: SparseVector(5, {0: 0.9641, 1: 0.0193, 2: 0.0029, 3: 0.0053, 4: 0.0084})
Step 2: Evaluate the Model Performance on Test Data
[In]: model_predictions = dec_tree_model.transform(test)
[In]: model_predictions.show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figf_HTML.jpg
We import RegressionEvaluation from MLlib, to evaluate the performance of the decision tree on test data. As of now, there are two metrics available for evaluation: r2 and RMSE (root mean squared error). r2 mainly suggests how much of the variation in the dataset can be attributed to regression. Therefore, the higher the r2, the better the performance of the model. On the other hand, RMSE suggests the total errors the model is making, in terms of the difference between actual and predicted values.
[In]: from pyspark.ml.evaluation import RegressionEvaluator
[In]: dt_evaluator = RegressionEvaluator(metricName='r2')
[In]: dt_r2 = dt_evaluator.evaluate(model_predictions)
[In]: print(f'The r-square value of DecisionTreeRegressor is {dt_r2}')
[Out]: The r-square value of DecisionTreeRegressor is 0.8093834699203476
[In]: dt_evaluator = RegressionEvaluator(metricName='rmse')
[In]: dt_rmse = dt_evaluator.evaluate(model_predictions)
[In]: print(f'The rmse value of DecisionTreeRegressor is {dt_rmse}')
[Out]: The rmse value of DecisionTreeRegressor is 0.014111932287681688

The r2 value of this particular model is close to 0.81, which is a little lower than that of a simple linear regression model.
Random Forest Regressors

Random forest regressors are a collection of multiple individual decision trees built using different samples of data. The whole idea of combining these individual trees is to take majority voting or averages (in case of regression) to generalize effectively. A random forest is, therefore, an ensembling technique that takes a bagging approach. It can be used for regression as well as for classification tasks. Because decision trees tend to overfit the data, random forests remove the element of high variance, by taking the means of the predicted values from individual trees. In our example, we will build a random forest model for regression, using default parameters (numTrees = 20)
Step 1: Build and Train Random Forest Regressor Model
[In]: from pyspark.ml.regression import RandomForestRegressor
[In]: rf = RandomForestRegressor()
[In]: rf_model = rf.fit(train)
[In]:  rf_model.featureImportances
[Out]: SparseVector(5, {0: 0.4395, 1: 0.045, 2: 0.0243, 3: 0.2725, 4: 0.2188})
As you can see, the number of trees in the random forest is equal to 20. This number can be increased.
[In]: rf_model.getNumTrees
[Out]: 20
[In]: model_predictions = rf_model.transform(test)
[In]: model_predictions.show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figg_HTML.jpg
Step 2: Evaluate the Model Performance on Test Data
We can again use r2and RMSE as the evaluation parameter of the random forest model.
[In]:rf_evaluator = RegressionEvaluator(metricName='r2')
[In]: rf_r2 = rf_evaluator.evaluate(model_predictions)
[In]: print(f'The r-square value of RandomForestRegressor is {rf_r2}')
[Out]: The r-square value of RandomForestRegressor is 0.8215863293044671
[In]: rf_evaluator = RegressionEvaluator(metricName='rmse')
[In]: rf_rmse = rf_evaluator.evaluate(model_predictions)
[In]: print(f'The rmse value of RandomForestRegressor is {rf_rmse}')
[Out]: The rmse value of RandomForestRegressor is 0.01365275410722947

As you can see, it clearly outperforms the decision tree regressor and has a higher r2. The performance of this model can further be enhanced with hyperparameter tuning.
Gradient-Boosted Tree Regressor

A gradient-boosted tree (GBT) regressor is also an ensembling technique, which uses boosting under the hood. Boosting refers to making use of individual weak learners in order to boost the performance of the overall model. One major difference between bagging and boosting is that in bagging, the individual models that are built are parallel in nature, meaning they can be built independent of each other, but in boosting, the individual models are built in a sequential manner. In a gradient boosting approach, the second model focuses on the errors made by the first model and tries to reduce overall errors for those data points. Similarly, the next model tries to reduce the errors made by the previous model. In this way, the overall error of prediction is reduced. In the following example, we will build a GBT regressor with default parameters.
Step 1: Build and Train a GBT Regressor Model
[In]: from pyspark.ml.regression import GBTRegressor
[In]: gbt = GBTRegressor()
[In]: gbt_model=gbt.fit(train)
[In]: gbt_model.featureImportances
[Out]: SparseVector(5, {0: 0.2325, 1: 0.2011, 2: 0.1645, 3: 0.2268, 4: 0.1751})
[In]: model_predictions = gbt_model.transform(test)
[In]: model_predictions.show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figh_HTML.jpg
Step 2: Evaluate the Model Performance on Test Data
[In]: gbt_evaluator = RegressionEvaluator(metricName='r2')
[In]: gbt_r2 = gbt_evaluator.evaluate(model_predictions)
[In]: print(f'The r-square value of GradientBoostedRegressor is {gbt_r2}')
[Out]: The r-square value of GradientBoostedRegressor is 0.8477273892307596
[In]: gbt_evaluator = RegressionEvaluator(metricName='rmse')
[In]: gbt_rmse = gbt_evaluator.evaluate(model_predictions)
[In]: print(f'The rmse value of GradientBoostedRegressor is {gbt_rmse}')
[Out]: The rmse value of GradientBoostedRegressor is 0.013305445803592103

As you can see, the GBT regressor outperforms the random forestmodel. With r2 being close to 0.85, it can be considered the final model, after proper tuning.
Building Multiple Models for Binary Classification Tasks

In this third and final section of the chapter, you will see how to build multiple machine learning models for binary classification tasks. The data that we are going to use for this is a subset of an open source Bank Marketing Data Set from the UCI ML repository, available at https://archive.ics.uci.edu/ml/datasets/Bank+Marketing.

There are two reasons for selecting only a subset of this data. The first is to maintain the class balance for the classification task, so as not to make it an anomalous detection category task. Another reason for selecting only a subset of the features is to limit the amount of signals in the data, as some of the features in the dataset strongly affect the output and, therefore, are ignored in this exercise.

The dataset contains 9,500 rows and 8 columns. The idea is to predict if the user will subscribe to another product or service (term deposit), based on the other attributes, such as age, job, loan, etc. This is a typical requirement in which machine learning is leveraged to find the top users who can be targeted by the business for cross-selling or upselling.

I’ll begin with the logistic regression model.
Logistic Regression

Logistic regression is considered to be one of baseline models, owing to its simplicity and interpretability. Under the hood, it is quite similar to linear regression. It also assumes that output is a linear combination of the dependent variables, but to keep the output between 0 and 1, as it returns the probability as output, it makes use of a nonlinear function (sigmoid), which produces an S curve instead of a straight line (linear regression).

We’ll start by building the baseline in Steps 1–3 and then complete the logistic regression model with default hyperparameters, in Steps 4–6.
Step 1: Read the Dataset
[In]: df=spark.read.csv('bank_data.csv',inferSchema=True,header=True)
[In]: df.count()
[Out]: 9501
[In]: df.columns
[Out]: ['age', 'job', 'marital', 'education', 'default', 'housing','loan', 'target_class']
[In]: df.printSchema()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figi_HTML.jpg
As you can see, the input columns are all the columns, except for the target class column. The target class is also well-balanced, in terms of the count of yes and no labels. We will have to convert yeses and noes into 1s and 0s, as well as rename the target_class column to “label,” which is the default acceptance column name in machine learning model parameters.
[In]: df.groupBy('target_class').count().show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figj_HTML.jpg
Step 2: Feature Engineering for Model
[In]: from pyspark.sql import functions as F
[In]: from pyspark.sql import ∗
[In]: df=df.withColumn("label", F.when(df.target_class =='no', F.lit(0)).otherwise(F.lit(1)))
[In]: df.groupBy('label').count().show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figk_HTML.jpg
Now that we have renamed the output column “label” and converted the target class to 1s and 0s, the next step is to create features for the model. Because we have categorical columns, such as job and edu, we will have to use StringIndexer and OneHotEncoder to convert them into a numerical format. We create a Python function, cat_to_num, to convert all the categorical features into numerical ones.
[In]: from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
[In]: def cat_to_num(df):
    for col in df.columns:
        stringIndexer = StringIndexer(inputCol=col, outputCol=col+"_index")
        model = stringIndexer.fit(df)
        indexed = model.transform(df)
        encoder = OneHotEncoder(inputCol=col+"_index", outputCol=col+"_vec")
        df = encoder.transform(indexed)
    df_assembler = VectorAssembler(inputCols=['age','marital_vec','education_vec','default_vec','housing_vec','loan_vec'], outputCol="features")
    df = df_assembler.transform(df)
    return df.select(['features','label'])
We just select the new features column and target label column, as we don’t need the earlier original columns for model training.
[In]: df_new=cat_to_num(df)
[In]: df_new.show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figl_HTML.jpg

Now we have all the input features merged into a single dense vector ('features'), along with output column labels, which we can use to train the machine learning models. The new dataframe created using only two columns (features, label) is now called df_new and will be used for every model. We can now split this new dataframe into train and test datasets. We can split the data into a 75%/25% ratio, using the randomplit function.
Step 3: Split the Data into Train and Test Datasets
[In]: train, test = df_new.randomSplit([0.75, 0.25])
[In]: print(f"Size of train Dataset : {train.count()}" )
[Out]: 7121
[In]: print(f"Size of test Dataset : {test.count()}" )
[Out]: 2380
Step 4: Build and Train the Logistic Regression Model
[In]: from pyspark.ml.classification import LogisticRegression
[In]: lr = LogisticRegression()
[In]: lr_model = lr.fit(train)
[In]:print( lr_model.coefficients)
[Out]:
[0.0272019114172,-0.647672064875,0.229030508111,-0.77074788287,-12.36869511,-12.8865599132,-13.2257790609,-12.6705131313,-13.0023164274,-13.0747662586,-12.6985757761,1.42220523957,0.301582233094,-0.0127231892838,0.218471149577,0.332362933568]

Once the model is built, we can make use of the internal function summary, which offers important details regarding the model, such as ROC curve, precision, recall, AUC (area under the curve), etc.
Step 5: Evaluate Performance on Training Data
[In]: lr_summary=lr_model.summary
[In]: lr_summary.accuracy
[Out]: 0.673079623648364
[In]: lr_summary.areaUnderROC
[Out]: 0.7186044694483512
[In]: lr_summary.weightedRecall
[Out]: 0.673079623648364
[In]: lr_summary.weightedPrecision
[Out]: 0.6750967624018298
Here, using the summary function , we can view the model’s performance on train data, such as its accuracy, AUC, weighted recall, and precision. We can also view additional details—such as how precision varies for various threshold values, the relation between precision and recall, and how recall varies with different threshold values—to pick the right threshold value for the model. These also can be plotted, to view the relationships.
[In]: lr_summary.precisionByThreshold.show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figm_HTML.jpg
[In]: lr_summary.roc.show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Fign_HTML.jpg
[In]: lr_summary.recallByThreshold.show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figo_HTML.jpg
[In]: lr_summary.pr.show()
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figp_HTML.jpg
Step 6: Evaluate Performance on Test Data
[In]: model_predictions = lr_model.transform(test)
[In]: model_predictions.columns
[Out]: ['features', 'label', 'rawPrediction', 'probability', 'prediction']
[In]: model_predictions.select(['label','probability','prediction']).show(10,False)
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figq_HTML.jpg
As you can see, the prediction column shows the model prediction for each of the records in the test data. The probability column shows the values for both classes (0 & 1). The probability at 0th index is of 0; the other is for a prediction of 1. The evaluation of the logistic regression model on test data can be done using BinaryClassEvaluator . We can get the area under ROC and that under the PR curve, as shown following:
[In]:from pyspark.ml.evaluation import BinaryClassificationEvaluator
[In]:lr_evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
[In]: lr_auroc = lr_evaluator.evaluate(model_predictions)
[In]: print(f'The auroc value of Logistic Regression Model is {lr_auroc}')
[Out]: The auroc value of Logistic Regression Model is 0.7092938229110143
[In]: lr_evaluator = BinaryClassificationEvaluator(metricName='areaUnderPR')
[In]: lr_aupr = lr_evaluator.evaluate(model_predictions)
[In]: print(f'The aupr value of Logistic Regression Model is {lr_aupr}')
[Out]: The aupr value of Logistic Regression Model is 0.6630743130940658
[In]: true_pos=model_predictions.filter(model_predictions['label']==1).filter(model_predictions['prediction']==1).count()
[In]: actual_pos=model_predictions.filter(model_predictions['label']==1).count()
[In]: pred_pos=model_predictions.filter(model_predictions['prediction']==1).count()
Recall
[In]: float(true_pos)/(actual_pos)
[Out]: 0.6701030927835051
Precision
[In]: float(true_pos)/(pred_pos)
[Out]: 0.6478405315614618
Decision Tree Classifier

As mentioned earlier, decision trees can be used for classification as well as regression. Here, we will build a decision tree with default hyperparameters and use it to predict whether the user will opt for the new term deposit plan.
Step 1: Build and Train Decision Tree Classifier Model
[In]: from pyspark.ml.classification import DecisionTreeClassifier
[In]: dt = DecisionTreeClassifier()
[In]: dt_model = dt.fit(train)
[In]: model_predictions = dt_model.transform(test)
[Out]: model_predictions.select(['label','probability','prediction']).show(10,False)
../images/478390_1_En_6_Chapter/478390_1_En_6_Figr_HTML.jpg
Step 2: Evaluate Performance on Test Data
[In]: dt_evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
[In]: dt_auroc = dt_evaluator.evaluate(model_predictions)
[In]: print(f'The auc value of Decision Tree Classifier Model is {dt_auroc}')
[Out]: The auc value of Decision Tree Classifier Model is 0.516199386190993
[In]: dt_evaluator = BinaryClassificationEvaluator(metricName='areaUnderPR')
[In]: dt_aupr = dt_evaluator.evaluate(model_predictions)
[In]: print(f'The aupr value of Decision Tree Model is {dt_aupr}')
[Out]: The aupr value of Decision Tree Model is 0.46771834172588167
[In]: true_pos=model_predictions.filter(model_predictions['label']==1).filter(model_predictions['prediction']==1).count()
[In]: actual_pos=model_predictions.filter(model_predictions['label']==1).count()
[In]: pred_pos=model_predictions.filter(model_predictions['prediction']==1).count()
[In]: float(true_pos)/(actual_pos)
[Out]: 0.6907216494845361
[In]: float(true_pos)/(pred_pos)
[Out]: 0.6661143330571665
Support Vector Machines Classifiers
Support vector machines (SVMs) are used for classification tasks, as they find the hyperplane that maximizes the margin (perpendicular distance) between two classes. All the instances and target classes are represented as vectors in high-dimensional space, and the SVM finds the closest two points from the two classes that support the best separating line or hyperplane, as shown in Figure 6-5.
../images/478390_1_En_6_Chapter/478390_1_En_6_Fig5_HTML.jpg
Figure 6-5

Support vector machine

For nonlinearly separable data, there are different kernel tricks to separate the classes. In our example, we will build a linearly separable support vector classifier with default hyperparameters.
Step 1: Build and Train SVM Model
[In]: from pyspark.ml.classification import LinearSVC
[In]: lsvc = LinearSVC()
[In]: lsvc_model = lsvc.fit(train)
[In]: model_predictions = lsvc_model.transform(test)
[In]: model_predictions.columns
[Out]: ['features', 'label', 'rawPrediction', 'prediction']
[In]:model_predictions.select(['label','prediction']).show(10,False)
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figs_HTML.jpg
Step 2: Evaluate Performance on Test Data
[In]: svc_evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
[In]: svc_auroc = svc_evaluator.evaluate(model_predictions)
[In]: print(f'The auc value of SupportVectorClassifier  is {svc_auroc}')
[Out]: The auc value of SupportVectorClassifier  is 0.7043772749366973
[In]: svc_evaluator = BinaryClassificationEvaluator(metricName='areaUnderPR')
[In]: svc_aupr =svc_evaluator.evaluate(model_predictions)
[In]: print(f'The aupr value of SupportVectorClassifier Model is {svc_aupr}')
[Out]: The aupr value of SupportVectorClassifier Model is 0.6567277377856992
[In]: true_pos=model_predictions.filter(model_predictions['label']==1).filter(model_predictions['prediction']==1).count()
[In]: actual_pos=model_predictions.filter(model_predictions['label']==1).count()
[In]: pred_pos=model_predictions.filter(model_predictions['prediction']==1).count()
[In]: float(true_pos)/(actual_pos)
[Out]: 0.7774914089347079
[In]: float(true_pos)/(pred_pos)
[Out]: 0.600132625994695
Naive Bayes Classifier

Naive Bayes (NB) classifiers work on the principle of conditional probability and assume absolute independence between predictors. An NB classifier doesn’t have many hyperparameters and can outperform some of the most sophisticated algorithms out there. In the following example, we will build an NB classifier and evaluate its performance on the test data.
Step 1: Build and Train SVM Model
[In]: from pyspark.ml.classification import NaiveBayes
[In]: nb = NaiveBayes()
[In]: nb_model = nb.fit(train)
[In]: model_predictions = nb_model.transform(test)
[In]: model_predictions.select(['label','probability','prediction']).show(10,False)
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figt_HTML.jpg
Step 2: Evaluate Performance on Test Data
[In]: nb_evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
[In]: nb_auroc = nb_evaluator.evaluate(model_predictions)
[In]: print(f'The auc value of NB Classifier is {nb_auroc}')
[Out]: The auc value of NB Classifier is 0.43543736717760884
[In]: nb_evaluator = BinaryClassificationEvaluator(metricName='areaUnderPR')
[In]: nb_aupr =nb_evaluator.evaluate(model_predictions)
[In]: print(f'The aupr value of NB Classifier Model is {nb_aupr}')
[Out]: The aupr value of NB Classifier Model is 0.4321001351769349
[In]: true_pos=model_predictions.filter(model_predictions['label']==1).filter(model_predictions['prediction']==1).count()
[In]: actual_pos=model_predictions.filter(model_predictions['label']==1).count()
[In]: pred_pos=model_predictions.filter(model_predictions['prediction']==1).count()
[In]: float(true_pos)/(actual_pos)
[Out]: 0.586
[In]: float(true_pos)/(pred_pos)
[Out]: 0.625
Gradient Boosted Tree Classifier

So far, we have used single algorithms for classification. Now we move on to use ensemble methods, such as GBT and random forests, for classification. Bagging and boosting for classification works according to similar principles as regression.
Step 1: Build and Train the GBT Model
[In]: from pyspark.ml.classification import GBTClassifier
[In]: gbt = GBTClassifier()
[In]: gbt_model = gbt.fit(train)
[In]: model_predictions = gbt_model.transform(test)
[In]: model_predictions.select(['label','probability','prediction']).show(10,False)
[Out]:
../images/478390_1_En_6_Chapter/478390_1_En_6_Figu_HTML.jpg
Step 2: Evaluate Performance on Test Data
[In]: gbt_evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
[In]: gbt_auroc = gbt_evaluator.evaluate(model_predictions)
[In]: print(f'The auc value of GradientBoostedTreesClassifier is {gbt_auroc}')
[Out]: The auc value of GradientBoostedTreesClassifier is 0.7392410330756018
[In]: gbt_evaluator = BinaryClassificationEvaluator(metricName='areaUnderPR')
[In]: gbt_aupr = gbt_evaluator.evaluate(model_predictions)
[In]: print(f'The aupr value of GradientBoostedTreesClassifier Model is {gbt_aupr}')
[Out]: The aupr value of GradientBoostedTreesClassifier Model is 0.7345982892755392
[In]: true_pos=model_predictions.filter(model_predictions['label']==1).filter(model_predictions['prediction']==1).count()
[In]: actual_pos=model_predictions.filter(model_predictions['label']==1).count()
[In]: pred_pos=model_predictions.filter(model_predictions['prediction']==1).count()
[In]: float(true_pos)/(actual_pos)
[Out]: 0.668
[In]: float(true_pos)/(pred_pos)
[Out]: 0.674


In [ ]:
Step 1: Build and Train the Random Forest Model
[In]: from pyspark.ml.classification import RandomForestClassifier
[In]: rf = RandomForestClassifier(numTrees=50,maxDepth=30)
[In]: rf_model = rf.fit(train)
[In]: model_predictions=rf_model.transform(test)
[In]: model_predictions.select(['label','probability','prediction']).show(10,False)

Step 2: Evaluate Performance on Test Data
[In]: rf_evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
[In]: rf_auroc = rf_evaluator.evaluate(model_predictions)
[In]: print(f'The auc value of RandomForestClassifier Model is {rf_auroc}')
[Out]:
The auc value of RandomForestClassifier Model is 0.7326433634020617
[In]: rf_evaluator = BinaryClassificationEvaluator(metricName='areaUnderPR')
[In]: rf_aupr = rf_evaluator.evaluate(model_predictions)
[In]: print(f'The aupr value of RandomForestClassifier Model is {rf_aupr}')
[Out]: The aupr value of RandomForestClassifier Model is 0.7277253895494864
[In]; true_pos=model_predictions.filter(model_predictions['label']==1).filter(model_predictions['prediction']==1).count()
[In]: actual_pos=model_predictions.filter(model_predictions['label']==1).count()
[In]: pred_pos=model_predictions.filter(model_predictions['prediction']==1).count()
[In]: float(true_pos)/(actual_pos)
[Out]: 0.67
[In]: float(true_pos)/(pred_pos)
[Out]: 0.67


In [ ]:
#Hyperparameter Tuning and Cross-Validation



[In]: from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
[In]: rf = RandomForestClassifier()
[In]: paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [5,10,20,25,30])
             .addGrid(rf.maxBins, [20, 60])
             .addGrid(rf.numTrees, [5, 20,50,100])
             .build())
[In]: cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=rf_evaluator, numFolds=5)
[In]: cv_model = cv.fit(train)
[In]: best_rf_model = cv_model.bestModel
best_rf_model contains the best hyper-parameters to be used for training the model on this dataset.
[In]: model_predictions = best_rf_model.transform(test)
[In]: rf_evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
[In]: rf_auroc = rf_evaluator.evaluate(model_predictions)
[In]: print(rf_auroc)
[Out]:  0.7425990374615659

In [33]:
#Unsupervised ML

df = spark.read.csv('/content/learn-pyspark/chap_7/music_data.csv',header=True,inferSchema=True)

In [37]:
df.show(3,truncate=False)

+----------------------------------------------------------------+----------------+
|user_id                                                         |music category  |
+----------------------------------------------------------------+----------------+
|05c388ffc3e28f2419fa8f4f0a4cdebfadb48a425bfc2a3a28115bc521911a17|Motivational    |
|05c388ffc3e28f2419fa8f4f0a4cdebfadb48a425bfc2a3a28115bc521911a17|Blues           |
|05c388ffc3e28f2419fa8f4f0a4cdebfadb48a425bfc2a3a28115bc521911a17|Electronic Music|
+----------------------------------------------------------------+----------------+
only showing top 3 rows



In [35]:
df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- music category: string (nullable = true)



In [38]:
import pyspark
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.clustering import KMeans

In [41]:
df.select('music category').distinct().count()

21

In [42]:
feature = df.stat.crosstab('user_id','music category')
feature.show(3)

+----------------------+-----------------+---------+-----+---------------+-------------+-----------+--------------+----------------+-----------+-------+---------+----+-----------+------------+-------+-----+-------+---+------+----+-------+
|user_id_music category|Alternative Music|Asian Pop|Blues|Classical Music|Country Music|Dance Music|Easy Listening|Electronic Music|Hindi Music|Hip Hop|Indie Pop|Jazz|Latin Music|Motivational|New Age|Opera|Popular|Rap|Reggae|Rock|Soulful|
+----------------------+-----------------+---------+-----+---------------+-------------+-----------+--------------+----------------+-----------+-------+---------+----+-----------+------------+-------+-----+-------+---+------+----+-------+
|  ca3c4285512798abe...|               65|        0|    7|              8|            7|         12|             7|               2|         12|      7|        0|  15|          0|           2|      0|    3|      0|  3|     0|   1|      0|
|  b457919ae168ac12e...|               80|  

In [45]:
from pyspark.ml.feature import VectorAssembler
feat_cols = [ col for col in feature.columns if col != 'user_id_music category']

In [46]:
vec_assembler = VectorAssembler(inputCols=feat_cols,outputCol='features').transform(feature)

In [49]:
vec_assembler.show(3)

+----------------------+-----------------+---------+-----+---------------+-------------+-----------+--------------+----------------+-----------+-------+---------+----+-----------+------------+-------+-----+-------+---+------+----+-------+--------------------+
|user_id_music category|Alternative Music|Asian Pop|Blues|Classical Music|Country Music|Dance Music|Easy Listening|Electronic Music|Hindi Music|Hip Hop|Indie Pop|Jazz|Latin Music|Motivational|New Age|Opera|Popular|Rap|Reggae|Rock|Soulful|            features|
+----------------------+-----------------+---------+-----+---------------+-------------+-----------+--------------+----------------+-----------+-------+---------+----+-----------+------------+-------+-----+-------+---+------+----+-------+--------------------+
|  ca3c4285512798abe...|               65|        0|    7|              8|            7|         12|             7|               2|         12|      7|        0|  15|          0|           2|      0|    3|      0|  3|  

In [50]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol='features',outputCol="scaledFeatures", 
                        withStd=True, withMean=False).fit(vec_assembler).transform(vec_assembler)

In [51]:
scaler.show(3)

+----------------------+-----------------+---------+-----+---------------+-------------+-----------+--------------+----------------+-----------+-------+---------+----+-----------+------------+-------+-----+-------+---+------+----+-------+--------------------+--------------------+
|user_id_music category|Alternative Music|Asian Pop|Blues|Classical Music|Country Music|Dance Music|Easy Listening|Electronic Music|Hindi Music|Hip Hop|Indie Pop|Jazz|Latin Music|Motivational|New Age|Opera|Popular|Rap|Reggae|Rock|Soulful|            features|      scaledFeatures|
+----------------------+-----------------+---------+-----+---------------+-------------+-----------+--------------+----------------+-----------+-------+---------+----+-----------+------------+-------+-----+-------+---+------+----+-------+--------------------+--------------------+
|  ca3c4285512798abe...|               65|        0|    7|              8|            7|         12|             7|               2|         12|      7|     

In [69]:
from pyspark.sql.functions import * 
from pyspark.sql.types import *
from pyspark.sql.functions import rand, randn
from pyspark.ml.clustering import KMeans
from pyspark.sql.functions import sha2
from pyspark.ml.evaluation import ClusteringEvaluator

In [72]:
errors=[]
for k in range(2,10):
  kmeans = KMeans(featuresCol='scaledFeatures',k=8)
  model = kmeans.fit(scaler)
  evaluator = ClusteringEvaluator()
  validate=evaluator.evaluate(model.transform(scaler))
  print(k,'-->',evaluator.evaluate(model.transform(scaler)))

2 --> 0.7901472797049975
3 --> 0.7901472797049975
4 --> 0.7901472797049975
5 --> 0.7901472797049975
6 --> 0.7901472797049975
7 --> 0.7901472797049975
8 --> 0.7901472797049975
9 --> 0.7901472797049975


In [73]:
kmeans6 = KMeans(featuresCol='scaledFeatures',k=6)
model_k6 = kmeans6.fit(scaler)
model_k6.transform(scaler).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|    1|
|         3|    1|
|         5|   38|
|         4|    4|
|         2|    4|
|         0|  727|
+----------+-----+



In [76]:
cluser_prediction=model_k6.transform(scaler)

In [77]:
from pyspark.ml.feature import PCA
pca = PCA(k=3, inputCol="scaledFeatures", outputCol="pca_features")
pca_model = pca.fit(cluser_prediction)

In [78]:
result = pca_model.transform(cluser_prediction).select('user_id_music category',"pca_features",'prediction')

In [79]:
result.show(truncate=False)

+----------------------------------------------------------------+----------------------------------------------------------------+----------+
|user_id_music category                                          |pca_features                                                    |prediction|
+----------------------------------------------------------------+----------------------------------------------------------------+----------+
|ca3c4285512798abe7d81fa2d0588549211be9516d9b2050e5564f8e912312c7|[-0.2250735026516283,0.15082893636632264,-0.13922132997808168]  |0         |
|b457919ae168ac12ec7e0303ae0d5fe292b63c9d04eb54e039942e7f46552ec4|[-0.5273003162834194,0.10206886950980101,-0.014971360310926076] |0         |
|a87bb168ba6e5a8da5eccce18c95565d843b40c1833b0f6c3089eb083f0fe3d3|[-0.23968261020295822,0.2801153618243113,-0.17382194436381926]  |0         |
|cf60c9b1a0444261c3a4687677f5336686b96756df5e33c11f6016f16fb38372|[-3.512051591993015,0.327294331895736,1.1017302617933258]       |0         |

In [80]:
clusters = result.toPandas().set_index('user_id_music category')

#Deep Learning Using PySpark

In [83]:
spark

In [84]:
import os
import numpy as np
import pandas as pd
from pyspark.sql.types import *


In [85]:
df = spark.read.csv('/content/learn-pyspark/chap_8/dl_data.csv',inferSchema=True,header=True)

In [86]:
df.show(3)

+-------------------+---------------------+-----------------+-------------------------------------+-------------------------------+-----------------------+------------------------------+-------------------------------+-------------------------+-------------------------+-------------------+------------------+-------------------+-----------------+------------------+------------------+-------------------------+
|Visit_Number_Bucket|Page_Views_Normalized|Orders_Normalized|Internal_Search_Successful_Normalized|Internal_Search_Null_Normalized|Email_Signup_Normalized|Total_Seconds_Spent_Normalized|Store_Locator_Search_Normalized|Mapped_Last_Touch_Channel|Mapped_Mobile_Device_Type|Mapped_Browser_Type|Mapped_Entry_Pages|Mapped_Site_Section|Mapped_Promo_Code|Maped_Product_Name|Mapped_Search_Term|Mapped_Product_Collection|
+-------------------+---------------------+-----------------+-------------------------------------+-------------------------------+-----------------------+---------------------

In [89]:
df.printSchema()

root
 |-- Visit_Number_Bucket: string (nullable = true)
 |-- Page_Views_Normalized: double (nullable = true)
 |-- Orders_Normalized: integer (nullable = true)
 |-- Internal_Search_Successful_Normalized: double (nullable = true)
 |-- Internal_Search_Null_Normalized: double (nullable = true)
 |-- Email_Signup_Normalized: double (nullable = true)
 |-- Total_Seconds_Spent_Normalized: double (nullable = true)
 |-- Store_Locator_Search_Normalized: double (nullable = true)
 |-- Mapped_Last_Touch_Channel: string (nullable = true)
 |-- Mapped_Mobile_Device_Type: string (nullable = true)
 |-- Mapped_Browser_Type: string (nullable = true)
 |-- Mapped_Entry_Pages: string (nullable = true)
 |-- Mapped_Site_Section: string (nullable = true)
 |-- Mapped_Promo_Code: string (nullable = true)
 |-- Maped_Product_Name: string (nullable = true)
 |-- Mapped_Search_Term: string (nullable = true)
 |-- Mapped_Product_Collection: string (nullable = true)



In [90]:
df = df.withColumnRenamed('Orders_Normalized','label')

In [91]:
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [93]:
train, validation, test  = df.randomSplit([0.7, 0.2, 0.1], 1234)

In [95]:
categorical_columns = [item[0] for item in df.dtypes if item[1].startswith('string')]
numeric_columns = [item[0] for item in df.dtypes if item[1].startswith('double')]

In [96]:
indexers = [StringIndexer(inputCol=column, outputCol='{0}_index'.format(column)) for column in categorical_columns]
featuresCreator = VectorAssembler(inputCols=[indexer.getOutputCol() for indexer in indexers] + numeric_columns, outputCol="features")

In [97]:
layers = [len(featuresCreator.getInputCols()), 4, 2, 2]
print(layers)

[16, 4, 2, 2]


In [98]:
classifier = MultilayerPerceptronClassifier(labelCol='label', featuresCol='features',maxIter=100, layers=layers, blockSize=128, seed=1234)

In [99]:
pipeline = Pipeline(stages=indexers + [featuresCreator, classifier])

In [100]:
model = pipeline.fit(train)

In [101]:
train_output_df = model.transform(train)
validation_output_df = model.transform(validation)
test_output_df = model.transform(test)

In [102]:
train_predictionAndLabels = train_output_df.select("prediction", "label")
validation_predictionAndLabels = validation_output_df.select("prediction", "label")
test_predictionAndLabels = test_output_df.select("prediction", "label")

In [103]:
metrics = ['weightedPrecision', 'weightedRecall', 'accuracy']

In [104]:
for metric in metrics:
    evaluator = MulticlassClassificationEvaluator(metricName=metric)
    print('Train ' + metric + ' = ' + str(evaluator.evaluate(train_predictionAndLabels)))
    print('Validation ' + metric + ' = ' + str(evaluator.evaluate(validation_predictionAndLabels)))
    print('Test ' + metric + ' = ' + str(evaluator.evaluate(test_predictionAndLabels)))

Train weightedPrecision = 0.9718465803511076
Validation weightedPrecision = 0.9724700548433042
Test weightedPrecision = 0.9699435168283953
Train weightedRecall = 0.9713846931436014
Validation weightedRecall = 0.9719613447665714
Test weightedRecall = 0.969260065288357
Train accuracy = 0.9713846931436014
Validation accuracy = 0.9719613447665714
Test accuracy = 0.969260065288357
